# Capstone: Pike Place Fish Market — Full Stack Agentic System

A single notebook that teaches **everything from A to Z**: system context, protocol architecture, 16 agentic patterns, and a complete implementation — first as mock, then as real MCP.

### What You'll Build

An AI system that buys fresh fish from Seattle's Pike Place Market. Five vendors compete on price, freshness, and reliability. Your agents discover options, learn from past purchases, negotiate prices, handle exceptions, and ask for human approval — using **every major agentic pattern** from the book.

### The Learning Arc

```
Part 1: System Context        ← Understand the protocol stack (no code)
Part 2: The Scenario           ← Meet the Pike Place vendors
Part 3: Infrastructure Classes ← Build the pattern building blocks
Part 4: Mock MCP               ← See all 16 patterns working (training wheels)
Part 5: Real MCP               ← Same patterns, production protocol (FastMCP + SQLite)
Part 6: Reflection             ← What we built, what's next
```

### Reference

This notebook maps to Antonio Gullí's *Agentic Design Patterns: A Hands-On Guide to Building Intelligent Systems* (2025), covering Chapters 2–8b plus new patterns (Memory, Learning, A2A, Goal Tracking, Exception Handling).

**Prerequisites:** Chapters 2 (Routing), 3 (Parallelization), 5 (Tool Use), 6 (Reflection), 7 (Multi-Agent), 8a (ReAct), 8b (Plan-and-Execute).

---
## Part 1: System Context — The Protocol Stack

Before we write a single line of code, let's understand the three protocols that power modern agentic systems and when to use each.

The AI community talks about "MCP vs A2A" and "MCP vs function calling" as if they're competitors. They're not. They're **three layers of one stack**, each solving a different problem at a different altitude. Understanding this is the difference between being a coder who follows tutorials and an architect who designs systems.

### The 3-Layer Protocol Stack

```
┌─────────────────────────────────────────────────────────┐
│                    YOUR AGENTIC SYSTEM                   │
├─────────────────────────────────────────────────────────┤
│                                                         │
│   Layer 3: A2A (Agent-to-Agent Protocol)                │
│   ┌─────────────┐         ┌─────────────┐              │
│   │ Buyer Agent  │◄──A2A──►│Vendor Agent │              │
│   └──────┬──────┘         └──────┬──────┘              │
│          │                       │                      │
│   Layer 2: MCP (Model Context Protocol)                 │
│   ┌──────┴──────┐         ┌──────┴──────┐              │
│   │ MCP Client   │◄─stdio─►│ MCP Server  │              │
│   │ (McpToolset) │  /SSE   │ (FastMCP)   │              │
│   └──────┬──────┘  /HTTP  └──────┬──────┘              │
│          │                       │                      │
│   Layer 1: Function Calling                             │
│   ┌──────┴──────┐         ┌──────┴──────┐              │
│   │     LLM      │◄──FC───►│ Application │              │
│   │  (Gemini)    │  (JSON) │   (ADK)     │              │
│   └─────────────┘         └─────────────┘              │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

Each layer talks to a **different partner**:

| Layer | Protocol | Who Talks | What They Say |
|-------|----------|-----------|---------------|
| 1 | **Function Calling** | LLM ↔ Application | "I want to use tool X with args Y" |
| 2 | **MCP** | Application ↔ Tool Server | "Here are my tools, their schemas, and how to call me" |
| 3 | **A2A** | Agent ↔ Agent | "Here's how we discover each other and collaborate" |

**Key insight:** Comparing MCP to function calling is like comparing HTTP to TCP — they're stacked, not competing. You need both to build a real system.

### Function Calling: How the LLM Talks to Your App

Function calling is **Layer 1** — the lowest layer. It's how the model says "I want to use a tool."

Every major LLM provider has their own format, but the concept is identical:

```json
// Gemini's function calling format
{
  "function_call": {
    "name": "get_market_snapshot",
    "args": {
      "fish_type": "King Salmon",
      "qty_lbs": 5.0
    }
  }
}
```

The model outputs structured JSON. Your application code (ADK, LangChain, etc.) reads the JSON, executes the function, and sends the result back to the model.

**What FC defines:**
- The format of tool requests (JSON schema)
- How tools are declared to the model (function descriptions)
- The request-response cycle between LLM and application

**What FC does NOT define:**
- Where the tool lives (same process? different machine?)
- How to discover tools at runtime
- How to maintain state between calls
- How to transport requests across process boundaries

That's where MCP comes in.

### MCP: How Tools Declare Themselves and Get Executed

MCP is **Layer 2** — the transport and discovery layer. It's NOT just another function calling format. It's the infrastructure that makes tools **reachable across process boundaries**.

**What MCP adds on top of function calling:**

| Capability | Function Calling | MCP |
|-----------|------------------|-----|
| Tool declaration | Hardcoded in prompt | Server declares at runtime (discovery) |
| Transport | In-process only | stdio, SSE, Streamable HTTP (cross-process) |
| State | None (stateless) | Server-side session state |
| Persistence | Application manages | SqliteSessionService, databases |
| Multi-client | Single LLM | Multiple clients can connect |

> **Transport evolution:** The original MCP spec offered `stdio` (local) and `SSE` (network). But SSE keeps a long-lived connection open — proxies timeout, load balancers kill idle streams, and TCP drops go undetected. The spec now recommends **Streamable HTTP**: each request is an independent `POST`, the server can respond with JSON or upgrade to a per-request SSE stream. For long-running agent tasks, `HTTP 202 Accepted` + polling is the most robust pattern — stateless, proxy-friendly, and recoverable. This notebook uses `stdio` (local subprocess), which sidesteps network concerns entirely.

> **"Isn't MCP just a fancy CLI?"** No. A CLI tool is a hammer — process starts, runs, exits, no memory. An MCP server is a workshop.
>
> | | CLI Tool | MCP Server |
> |---|---------|-----------|
> | Discovery | You read `--help` | Client calls `tools/list` at runtime — gets names, schemas, descriptions |
> | Schema | Freeform strings in/out | JSON Schema per parameter. The LLM knows exactly what to send |
> | State | Stateless. Every call starts from zero | Server holds session state across calls |
> | Lifecycle | Process starts, runs, exits | Server stays up, serves many clients, maintains DB connections |
> | Composability | Pipe `stdout` and pray | Structured JSON-RPC. 8 tools on one server, each building on the last |
>
> Could you wrap every CLI tool in a shell script? Sure — the same way you could serve a website with `netcat`. The question is whether it scales to 8 tools, 3 agents, and a 15-minute conversation.

**Pike Place example:**

Your FastMCP server exposes 8 tools. Any LLM — Gemini, Claude, GPT — can connect to it, discover the tools, and call them. The server maintains purchase history in SQLite that survives process restarts.

```
Client (any LLM)                Server (FastMCP)
      │                               │
      ├── "list tools" ───────────►   │
      │   ◄── [get_market_snapshot,   │
      │        negotiate_price, ...]  │
      │                               │
      ├── "call get_market_snapshot"──►│
      │   ◄── {vendors: [...]}        │
      │                               │
      │   (Server remembers this      │
      │    interaction in SQLite)      │
```

The client doesn't know or care how the server implements its tools. It just knows the schemas.

### MCP vs Function Calling — The Real Comparison

This is the comparison people actually debate. Here's the clear answer:

| | Function Calling | MCP |
|---|---|---|
| **What it is** | Request format | Transport + discovery protocol |
| **Who defines it** | LLM provider (Gemini, OpenAI) | Open standard (Anthropic-originated) |
| **Where tools live** | Same process as the app | Anywhere (local, remote, cloud) |
| **Discovery** | Static (defined at prompt time) | Dynamic (server declares at runtime) |
| **State** | None | Server can maintain sessions |
| **Analogy** | The envelope format | The postal service |

**They work TOGETHER, not against each other:**

```
1. Gemini uses Function Calling to say:
   "I want to call get_market_snapshot with fish_type='King Salmon'"

2. ADK's McpToolset uses MCP to:
   - Route that request over stdio to the FastMCP server
   - Server executes the function
   - Server sends result back over stdio

3. ADK gives the result back to Gemini via Function Calling response
```

The LLM doesn't know MCP exists. MCP doesn't know which LLM is calling. They're separate layers that compose cleanly.

### Context Extension: What MCP Actually Gives You

The term "context extension" captures what MCP really is — it **extends your LLM's capabilities** without retraining or re-prompting.

Without MCP, your LLM can only use tools that are:
- Defined in the same codebase
- Running in the same process
- Stateless (no memory between calls)

With MCP, your LLM gains:

**1. Runtime Discovery**
The LLM connects to a server and asks "what can you do?" The server responds with tool schemas. Add a new tool to the server → every connected client can use it immediately. No code changes on the client.

**2. Transport Independence**
Same server, three ways to connect:
- `stdio` — subprocess (what we use: simple, local)
- `SSE` — server-sent events (network, streaming)
- `HTTP` — REST-style (network, stateless)

**3. Server-Side Persistence**
Our FastMCP server uses ADK's `SqliteSessionService`. Purchase history survives subprocess restarts. The client agent dies and respawns every phase — but the server's memory endures.

**4. Cross-Process State**
The server can maintain negotiation state, vendor scores, purchase history — none of which the client needs to know about. The client just calls tools and gets results.

This is why "MCP vs function calling" is the wrong framing. MCP doesn't replace function calling — it **extends** what function calling can reach.

### A2A: When Tools Become Agents

A2A (Agent-to-Agent Protocol) is **Layer 3** — the collaboration layer. It answers the question: "How do autonomous agents discover each other and work together?"

**The key distinction:**

| | MCP | A2A |
|---|---|---|
| **Message to server** | "Call this function for me" | "Talk to this agent for me" |
| **Server behavior** | Executes deterministic code | **Reasons** about the request |
| **Server has LLM?** | No — pure code | Yes — its own model |
| **Interaction pattern** | Request → Response | Conversation (multi-turn) |

**Pike Place example:**

Right now, our `negotiate_price` tool is **MCP** — the server applies deterministic midpoint math:
```
offer >= list_price → accept
offer >= min_price  → counter at midpoint
offer < min_price   → reject
```
No reasoning. No strategy. Just math.

With **A2A**, the vendor would have its own LLM agent that thinks:
- "Inventory is running low — I won't budge on price"
- "This buyer bought from me last week — I'll offer a loyalty discount"
- "The halibut vendor is offering a better deal — I need to be competitive"

The buyer agent and vendor agent would have a **conversation**, not a function call. That's A2A.

### MCP vs A2A — Completing the Picture

Now we can see the full stack clearly:

| Layer | Protocol | Partners | Server Has LLM? | Example |
|-------|----------|----------|-----------------|---------|
| 1 | **Function Calling** | LLM ↔ App | N/A | Gemini outputs `{"function_call": "get_market_snapshot"}` |
| 2 | **MCP** | App ↔ Tool Server | **No** | FastMCP server executes `get_market_snapshot()` and returns data |
| 3 | **A2A** | Agent ↔ Agent | **Yes** | Vendor agent *reasons* about pricing strategy and responds |

**Common misconception:** "Should I use MCP or A2A?"

Wrong question. They solve different problems:
- Use **MCP** when the server-side task is **deterministic** — database queries, API calls, calculations, data retrieval
- Use **A2A** when the server-side task requires **reasoning** — negotiation strategy, creative responses, contextual decision-making
- Most systems use **both** — MCP for data tools, A2A for agent-to-agent collaboration

**In this notebook:**
- Parts 3-5 demonstrate **MCP** (tools that execute code)
- The NegotiationProtocol class is a **stepping stone toward A2A** — structured messages, but deterministic responses
- Part 6 sketches what the full **A2A upgrade** would look like

### Design Principles: What Lives Where?

The most important architectural decision in any agentic system: **what logic lives on the server vs the client?**

| | **Server Owns** | **Client Owns** |
|---|---|---|
| **State** | Purchase history, vendor scores, learning data | Nothing — stateless |
| **Persistence** | SqliteSessionService, databases | Ephemeral (per-run) |
| **Business Logic** | Negotiation math, price calculations, freshness checks | Orchestration, phase sequencing |
| **Data** | Vendor inventory, prices, cost basis | Only what MCP tools return |
| **Learning** | Learning inference, best-vendor computation | Goal checking, display |
| **Protocol** | Negotiation protocol state | Human approval gate |
| **Survives restart?** | ✅ Yes (SQLite) | ❌ No (dies with process) |

**Why this split?**

1. **Server survives restarts.** The client agent spawns, runs, dies — every phase. But the server's SQLite database persists. Purchase history from Scenario 1 is available in Scenario 2.

2. **Client stays swappable.** Swap Gemini for Claude? Change the `MODEL` variable. The server doesn't change. All the business logic, data, and persistence are behind MCP tools.

3. **Separation of concerns.** The LLM is good at reasoning ("which tool should I call next?"). Code is good at executing ("compute the midpoint price"). Don't make the LLM do math. Don't make code do reasoning.

### Anti-Patterns: Common Mistakes

| Anti-Pattern | Problem | Fix |
|-------------|---------|-----|
| **Chatty client** | Client makes 10 individual MCP calls when 1 batch call works | Add batch tools like `get_market_snapshot` (one call, all vendor data) |
| **Thick client storing state** | Client keeps purchase history in Python variables — lost on restart | Move state to server with SqliteSessionService |
| **Server with LLM for deterministic tasks** | Using an LLM to compute price midpoints (waste of tokens) | Server does math in code, reserve LLMs for reasoning |
| **No error boundary** | Server error crashes client pipeline | Client catches MCP errors, logs via ExceptionHandler, tries next vendor |
| **Reusing agents for wrong tasks** | Using BuyerAgent (10 tools) to call 1 learning tool | Create focused LearnerAgent with 1 tool and strict instructions |
| **Recording wrong data** | Storing target_price instead of actual negotiated price | Parse LLM output for real price, not the input |

Every one of these is a real bug we found and fixed while building this notebook.

### Pattern Catalog — 16 Patterns Mapped to Book

| # | Pattern | Book Chapter | Category | What It Does | This Notebook |
|---|---------|-------------|----------|-------------|---------------|
| 1 | Routing | Ch 2 | Internal | Extracts intent, routes to specialist agent | Part 4: Coordinator in orchestrator |
| 2 | Parallelization | Ch 3 | Internal | Queries multiple vendors concurrently | Part 4: `query_vendor_catalog` checks all 5 |
| 3 | Tool Use | Ch 5 | FC + MCP | Agents call external functions | Part 4: FunctionTool, Part 5: McpToolset |
| 4 | Reflection | Ch 6 | Internal | QualityInspector rejects low-freshness picks | Part 4: Inspector agent |
| 5 | Multi-Agent | Ch 7 | Internal | Specialized agents with distinct roles | Parts 4 & 5: Buyer, Inspector, Negotiator, Recorder |
| 6 | Sequential Pipeline | Ch 7 | Internal | Discovery → Inspection → Negotiation → Record | Parts 4 & 5: Orchestrator phases |
| 7 | AgentTool | Ch 7 | MCP | Vendor MCP server used as tool by negotiator | Part 5: McpToolset wraps server |
| 8 | ReAct | Ch 8a | Internal | BuyerAgent reasons between tool calls | Part 4: iterative vendor evaluation |
| 9 | Plan-and-Execute | Ch 8b | Internal | Python orchestrator drives the loop, replans on failure | Parts 4 & 5: CapstoneOrchestrator / MCPOrchestrator |
| 10 | Memory Management | New | Internal→MCP | Stores purchase history, vendor reliability scores | Part 3: PurchaseMemory, Part 5: SqliteSessionService |
| 11 | Learning & Adaptation | New | Internal→MCP | Finds best vendors, predicts prices from history | Part 3: LearningEngine, Part 5: `run_learning_inference` |
| 12 | Goal Tracking | New | Internal | Tracks budget, freshness, deadline constraints | Part 3: GoalTracker (stays on client) |
| 13 | Exception Handling | New | Internal | Logs failures, recommends recovery strategies | Part 3: ExceptionHandler (stays on client) |
| 14 | A2A Protocol | New | A2A | Structured offer/counter-offer between buyer and vendor | Part 3: NegotiationProtocol |
| 15 | MCP Server | New | MCP | Vendor tools behind standardized protocol | Part 4: MockMCPVendorServer, Part 5: FastMCP |
| 16 | Human-in-the-Loop | New | Internal | Customer approves before purchase | Parts 4 & 5: Approval gate |

### How the Patterns Connect

The patterns aren't isolated — they form a **dependency graph**:

```
                    ┌──────────────┐
                    │   Customer   │
                    │   Request    │
                    └──────┬───────┘
                           │
                    ┌──────▼───────┐
                    │   Routing    │ Ch 2: extract fish_type, qty, budget
                    └──────┬───────┘
                           │
                    ┌──────▼───────┐
          ┌────────│Parallelization│ Ch 3: query all vendors
          │        └──────┬───────┘
          │               │
   ┌──────▼───────┐ ┌────▼────────┐
   │   Memory     │ │  Tool Use   │ Ch 5: MCP calls to vendor servers
   │  (history)   │ └─────┬───────┘
   └──────┬───────┘       │
          │         ┌─────▼───────┐
   ┌──────▼───────┐ │ Reflection  │ Ch 6: inspector rejects bad options
   │  Learning    │ └─────┬───────┘
   │ (best vendor)│       │
   └──────┬───────┘ ┌─────▼───────┐
          │         │ A2A Protocol │ Negotiation: offer/counter/accept
          └────────►│ (Negotiate)  │
                    └─────┬───────┘
                          │
                   ┌──────▼───────┐
                   │Goal Tracking  │ Budget/freshness/deadline check
                   └──────┬───────┘
                          │
                   ┌──────▼───────┐
                   │   Human      │ "Approve $139.50 for 5lb King Salmon?"
                   │  Approval    │
                   └──────┬───────┘
                          │
                   ┌──────▼───────┐
                   │   Record     │ Save to memory → feeds future Learning
                   │  Purchase    │
                   └──────┬───────┘
                          │
                   ┌──────▼───────┐
                   │  Exception   │ If anything fails → log, recover, retry
                   │  Handler     │
                   └──────────────┘
```

**Memory feeds Learning. Learning guides Routing. Goals gate Negotiation. Protocol structures A2A. Exceptions handle failures.** Five classes, one pipeline.

### The Learning Journey in This Notebook

This notebook is structured as a **progression**, not a reference:

| Part | What You Learn | Key Insight |
|------|---------------|-------------|
| **Part 2: Scenario** | The problem domain | Fish market as a microcosm of real-world agent challenges |
| **Part 3: Infrastructure** | The pattern building blocks | 5 classes, each implements one pattern from the book |
| **Part 4: Mock MCP** | How patterns compose | All 16 patterns working together, visible tool calls, transparent reasoning |
| **Part 5: Real MCP** | How to go from mock to production | Same outcome, real protocol. Server owns state, client stays light |
| **Part 6: Reflection** | Architectural tradeoffs | When to use what, thin vs thick server, where A2A fits |

**Part 4 and Part 5 produce the same result.** That's intentional. The mock lets you see the plumbing. The real MCP shows you the production architecture. Same fish, same vendors, same prices — different protocol layer.

If you only have time for one section, read Part 1 (you're doing that now) and run Part 4. If you want the full picture, go through all six parts.

### Key Questions This Notebook Answers

By the end of this notebook, you'll be able to answer:

**Protocol Architecture**
- When should I use MCP vs native function calling?
- When does a tool become an agent (MCP → A2A)?
- What does "context extension" actually mean?

**System Design**
- Should business logic live on the server or client?
- How do I make agent memory survive process restarts?
- When is a chatty client a bug vs a feature?

**Pattern Composition**
- How do 16 patterns compose into one coherent system?
- Which patterns stay on the client when I add MCP?
- What's the minimum viable agent (thin client)?

**Production Readiness**
- How do I migrate from mock to real MCP?
- What changes and what stays the same?
- Where does this notebook end and production begin?

Let's start building.

---

## Part 2: Pike Place Fish Market — The Scenario

You're ordering fresh fish from Seattle's Pike Place Market for a dinner party this Saturday. Five vendors compete on price, freshness, and reliability. Your AI system needs to:

1. **Discover** what's available across all vendors
2. **Learn** from past purchases to pick the best vendor
3. **Negotiate** prices using structured offers and counter-offers
4. **Check** that the deal meets your budget, freshness, and delivery constraints
5. **Handle** exceptions when things go wrong (out of stock, price spikes)
6. **Ask you** before spending your money
7. **Remember** the outcome for next time

All data is **deterministic mock** — same results every run, same principle as Chapters 6 and 8a. In production, you'd replace mock data with live APIs.

### The Vendor Ecosystem

| Vendor | Specialty | Price Range | Freshness | Reliability |
|--------|-----------|-------------|-----------|-------------|
| **Wild Salmon Co** | Premium King Salmon | $28-32/lb | 9-10/10 | 95% on-time |
| **Atlantic Fishery** | Farmed Salmon | $16-20/lb | 6-7/10 | 90% on-time |
| **Halibut House** | Pacific Halibut | $32-38/lb | 8-9/10 | 85% on-time |
| **Tuna King** | Yellowfin Tuna | $20-25/lb | 7-8/10 | 92% on-time |
| **Crab Cart** | Dungeness Crab | $14-18/lb | 8-9/10 | 88% on-time |

**Why these 5?** Each creates a different tradeoff:
- Wild Salmon Co vs Atlantic Fishery — expensive+fresh vs cheap+stale (quality/price tradeoff)
- Halibut House — high quality but lower reliability (risk tradeoff)
- Multiple vendors carry the same fish — forces the system to compare and learn preferences

In [1]:
import os
import json
import time
import re
import nest_asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any

nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.genai import types
from google.genai import types as genai_types

# ── ADK-native retry config (Ch 8b lesson) ────────────────
RETRY_CONFIG = genai_types.GenerateContentConfig(
    http_options=genai_types.HttpOptions(
        retry_options=genai_types.HttpRetryOptions(
            initial_delay=2,
            attempts=3,
        ),
    ),
)

MODEL = "gemini-2.0-flash"
print(f"Setup complete: {MODEL} with auto-retry on 429s")

Setup complete: gemini-2.0-flash with auto-retry on 429s


In [2]:
# ── Pike Place Vendor Data ───────────────────────────────
# This data drives ALL scenarios in Parts 3-5.
# Each vendor has: inventory, pricing, freshness scores, reliability,
# cost_basis (for negotiation margin calculations), and min_margin.

VENDORS = {
    "Wild Salmon Co": {
        "specialty": ["King Salmon", "Sockeye Salmon", "Coho Salmon"],
        "inventory": {"King Salmon": 25, "Sockeye Salmon": 40, "Coho Salmon": 15},
        "price_per_lb": {"King Salmon": 28, "Sockeye Salmon": 22, "Coho Salmon": 19},
        "freshness_score": {"King Salmon": 9, "Sockeye Salmon": 10, "Coho Salmon": 8},
        "reliability": 0.95,
        "min_margin": 0.20,
        "cost_basis": {"King Salmon": 18, "Sockeye Salmon": 14, "Coho Salmon": 12},
    },
    "Atlantic Fishery": {
        "specialty": ["King Salmon", "Atlantic Salmon"],
        "inventory": {"King Salmon": 50, "Atlantic Salmon": 100},
        "price_per_lb": {"King Salmon": 18, "Atlantic Salmon": 14},
        "freshness_score": {"King Salmon": 6, "Atlantic Salmon": 7},
        "reliability": 0.90,
        "min_margin": 0.15,
        "cost_basis": {"King Salmon": 12, "Atlantic Salmon": 9},
    },
    "Halibut House": {
        "specialty": ["Pacific Halibut", "Lingcod"],
        "inventory": {"Pacific Halibut": 12, "Lingcod": 20},
        "price_per_lb": {"Pacific Halibut": 35, "Lingcod": 24},
        "freshness_score": {"Pacific Halibut": 9, "Lingcod": 8},
        "reliability": 0.85,
        "min_margin": 0.25,
        "cost_basis": {"Pacific Halibut": 22, "Lingcod": 15},
    },
    "Tuna King": {
        "specialty": ["Yellowfin Tuna", "Albacore Tuna"],
        "inventory": {"Yellowfin Tuna": 30, "Albacore Tuna": 45},
        "price_per_lb": {"Yellowfin Tuna": 22, "Albacore Tuna": 16},
        "freshness_score": {"Yellowfin Tuna": 8, "Albacore Tuna": 7},
        "reliability": 0.92,
        "min_margin": 0.18,
        "cost_basis": {"Yellowfin Tuna": 14, "Albacore Tuna": 10},
    },
    "Crab Cart": {
        "specialty": ["Dungeness Crab", "King Crab Legs"],
        "inventory": {"Dungeness Crab": 35, "King Crab Legs": 8},
        "price_per_lb": {"Dungeness Crab": 16, "King Crab Legs": 45},
        "freshness_score": {"Dungeness Crab": 9, "King Crab Legs": 7},
        "reliability": 0.88,
        "min_margin": 0.22,
        "cost_basis": {"Dungeness Crab": 10, "King Crab Legs": 30},
    },
}

print(f"Pike Place Market: {len(VENDORS)} vendors loaded")
for name, v in VENDORS.items():
    print(f"  {name}: {', '.join(v['specialty'])}")

Pike Place Market: 5 vendors loaded
  Wild Salmon Co: King Salmon, Sockeye Salmon, Coho Salmon
  Atlantic Fishery: King Salmon, Atlantic Salmon
  Halibut House: Pacific Halibut, Lingcod
  Tuna King: Yellowfin Tuna, Albacore Tuna
  Crab Cart: Dungeness Crab, King Crab Legs


### How Patterns Map to This Scenario

| Pattern | Pike Place Example |
|---------|--------------------|
| **Routing** | Extract "King Salmon, 5lb, $150 budget" from natural language request |
| **Parallelization** | `query_vendor_catalog` checks all 5 vendors in one call |
| **Tool Use** | 7 tools: catalog, freshness, shipping, negotiate, learning, record, goals |
| **Reflection** | QualityInspector rejects Atlantic Fishery (freshness 6 < 8 for sashimi) |
| **Multi-Agent** | BuyerAgent, QualityInspector, Negotiator, Recorder — each with one job |
| **Sequential Pipeline** | Discovery → Inspection → Learning → Negotiation → Approval → Record |
| **ReAct** | BuyerAgent reasons between tool calls: "Freshness 6 is too low, try next vendor" |
| **Plan-and-Execute** | Orchestrator drives the loop in Python (not LLM), replans on failure |
| **Memory** | PurchaseMemory stores every completed purchase for future reference |
| **Learning** | LearningEngine computes: "Wild Salmon Co has best quality/price ratio" |
| **Goal Tracking** | GoalTracker flags: budget AT_RISK when total hits $139.50 of $150 |
| **Exception Handling** | OUT_OF_STOCK at Wild Salmon → try Atlantic Fishery as fallback |
| **A2A Protocol** | OFFER $23.80 → COUNTER $25.90 → ACCEPT $25.90 (structured messages) |
| **MCP Server** | FastMCP exposes tools via stdio — client doesn't know server internals |
| **Human-in-the-Loop** | "Approve $139.50 for 5lb King Salmon from Wild Salmon Co?" → Yes |

---

---
## Part 3: Infrastructure Classes — The Building Blocks

These 5 classes implement core patterns from the book. They're used in **both** Part 4 (mock MCP) and Part 5 (real MCP).

| Class | Pattern | Purpose |
|-------|---------|---------|
| `PurchaseMemory` | Memory Management | Stores past purchases, vendor reliability scores |
| `LearningEngine` | Learning & Adaptation | Finds best vendors, predicts prices from history |
| `GoalTracker` | Goal Setting & Monitoring | Tracks budget, freshness, deadline constraints |
| `ExceptionHandler` | Exception Handling | Logs failures, recommends recovery strategies |
| `NegotiationProtocol` | Agent-to-Agent (A2A) | Structured offer/counter-offer between buyer and vendor |

In Part 4 (mock), these classes run in-process. In Part 5 (real MCP), PurchaseMemory and LearningEngine get replaced by ADK's `SqliteSessionService` on the server.

In [3]:
# ── Memory Management ─────────────────────────────────
# Pattern: Memory Management (persists across scenarios)
# Purpose: The agent's "experience database." Every completed purchase
#          gets stored here so the LearningEngine can analyze it later.
#          The system literally gets smarter with each purchase.

class PurchaseMemory:
    """Stores past purchases and vendor reliability scores.
    
    Two data structures, two purposes:
      - history:       append-only log of every purchase (the raw facts)
      - vendor_scores: rolling reliability scores per vendor, used to
                       rank vendors before we even check prices
    
    Design choice: append-only (no edits, no deletes). This mirrors how
    real learning works — you don't forget bad experiences, you learn from them.
    """

    def __init__(self):
        self.history: List[Dict] = []
        self.vendor_scores: Dict[str, List[float]] = {}  # vendor -> [score1, score2, ...]

    def add_purchase(self, vendor: str, fish_type: str, qty: float,
                     price_per_lb: float, freshness: int, on_time: bool):
        """Record a completed purchase and update vendor reliability.
        
        Called by the orchestrator AFTER human approval + successful deal.
        Builds a complete record with derived fields (total_cost, timestamp)
        so downstream queries don't need to recompute them.
        """
        record = {
            "vendor": vendor, "fish_type": fish_type, "qty": qty,
            "price_per_lb": price_per_lb,
            "total_cost": qty * price_per_lb,      # derived — convenience for reports
            "freshness": freshness, "on_time": on_time,
            "timestamp": datetime.now().isoformat(),
        }
        self.history.append(record)

        # Reliability scoring:
        #   freshness / 10  → normalizes to 0-1 range
        #   on_time penalty → 1.0 if on time, 0.5 if late (harsh but fair)
        # Examples:
        #   freshness=9, on_time=True  → 0.9  (great vendor)
        #   freshness=6, on_time=False → 0.3  (avoid this vendor)
        score = (freshness / 10) * (1.0 if on_time else 0.5)
        self.vendor_scores.setdefault(vendor, []).append(score)
        return record

    def get_vendor_reliability(self, vendor: str) -> float:
        """Average reliability across all purchases from this vendor.
        
        Returns 0.5 for unknown vendors — deliberately neutral, not pessimistic.
        This prevents the system from refusing to try new vendors it hasn't
        seen before. A vendor earns its way up (or down) from 0.5.
        """
        scores = self.vendor_scores.get(vendor, [])
        return sum(scores) / len(scores) if scores else 0.5

    def get_history_for(self, fish_type: str) -> List[Dict]:
        """Filter: all past purchases of a specific fish type.
        
        Why per-fish (not per-vendor)? Because a vendor might be excellent
        for salmon but terrible for crab. The LearningEngine needs fish-level
        granularity to make accurate recommendations.
        """
        return [h for h in self.history if h["fish_type"] == fish_type]

    def summary(self) -> str:
        """Human-readable dump of memory state.
        
        Shows total transaction count + per-vendor reliability average.
        Used for debugging and the 'Final System State' output cell.
        """
        lines = [f"Purchase history: {len(self.history)} transactions"]
        for vendor, scores in self.vendor_scores.items():
            avg = sum(scores) / len(scores)
            lines.append(f"  {vendor}: {len(scores)} purchases, reliability {avg:.2f}")
        return "\n".join(lines)


# Global memory instance — persists across all phases
memory = PurchaseMemory()
print("PurchaseMemory initialized (empty)")

PurchaseMemory initialized (empty)


In [4]:
# ── Learning & Adaptation ─────────────────────────────
# Pattern: Learning & Adaptation
# Purpose: Sits ON TOP of PurchaseMemory and answers the question:
#          "Given what we know, what should we do differently this time?"
#          It doesn't store anything itself — it computes insights on-the-fly.

class LearningEngine:
    """Learns from past purchases to improve future decisions.
    
    Key design: dependency injection. The engine reads from PurchaseMemory
    but owns no data. You could swap in a different memory backend (SQLite,
    Redis, ADK SqliteSessionService) without touching learning logic.
    """

    def __init__(self, mem: PurchaseMemory):
        self.memory = mem

    def best_vendor_for(self, fish_type: str) -> Optional[str]:
        """Which vendor delivered the best quality/price ratio for this fish?
        
        Algorithm:
          1. Pull all purchases of this fish_type from memory
          2. For each purchase, compute: freshness / price_per_lb
             - This is a "bang for your buck" score
             - High freshness + low price = high score
             - Low freshness + high price = low score
          3. Average scores per vendor (handles repeat purchases)
          4. Return the vendor with the highest average score
        
        Why freshness/price and not just cheapest?
          A $18/lb fish with freshness 6 → score 0.33
          A $28/lb fish with freshness 9 → score 0.32
          Nearly equal! But the $28 fish is sashimi-grade.
          The ratio captures VALUE, not just cost.
        
        max(price_per_lb, 1) prevents division-by-zero if price is
        somehow recorded as 0 (defensive coding).
        
        Returns None if no history — signals "first-time buy" to the orchestrator.
        """
        history = self.memory.get_history_for(fish_type)
        if not history:
            return None
        # Score = freshness / price_per_lb (higher is better)
        vendor_scores = {}
        for h in history:
            v = h["vendor"]
            score = h["freshness"] / max(h["price_per_lb"], 1)
            vendor_scores.setdefault(v, []).append(score)
        avg_scores = {v: sum(s)/len(s) for v, s in vendor_scores.items()}
        return max(avg_scores, key=avg_scores.get)

    def avg_price(self, fish_type: str) -> Optional[float]:
        """What's the average price we've historically paid for this fish?
        
        Used by the orchestrator to set negotiation targets. If we've
        been paying $25/lb on average, we know to start our opening
        offer below that (e.g., $21/lb).
        
        Returns None if no history — can't average zero purchases.
        """
        history = self.memory.get_history_for(fish_type)
        if not history:
            return None
        return sum(h["price_per_lb"] for h in history) / len(history)

    def get_insights(self, fish_type: str) -> str:
        """The 'report card' — a human-readable summary of what we've learned.
        
        Called by the BuyerAgent tool (get_learning_insights) BEFORE
        searching for vendors. Returns:
          - How many past purchases we have
          - Best vendor (from best_vendor_for)
          - Average price paid (our historical baseline)
          - Best freshness ever seen (sets quality expectations)
        
        If no history: returns 'first time buying' message.
        This is what triggers the 'no history' vs 'learning active'
        indicator in the orchestrator output.
        """
        best = self.best_vendor_for(fish_type)
        avg = self.avg_price(fish_type)
        history = self.memory.get_history_for(fish_type)
        if not history:
            return f"No purchase history for {fish_type}. This is our first time buying it."
        return (
            f"Learning insights for {fish_type}:\n"
            f"  Past purchases: {len(history)}\n"
            f"  Best vendor: {best}\n"
            f"  Average price: ${avg:.2f}/lb\n"
            f"  Best freshness seen: {max(h['freshness'] for h in history)}/10"
        )


learning = LearningEngine(memory)
print("LearningEngine initialized (linked to PurchaseMemory)")

LearningEngine initialized (linked to PurchaseMemory)


In [5]:
# ── Goal Setting & Monitoring ────────────────────────
# Pattern: Goal Setting & Monitoring
# Purpose: The system's constraint checker. Before any purchase is approved,
#          GoalTracker answers: "Does this deal violate any of the customer's
#          requirements?" It REPORTS but doesn't DECIDE — the orchestrator
#          reads the report and acts on it.

class GoalTracker:
    """Monitors purchase goals and flags violations.
    
    Three hard constraints set at pipeline start:
      - budget:        max total spend across all purchases in this session
      - min_freshness: minimum acceptable freshness score (1-10 scale)
      - delivery_by:   deadline (used for display; delivery_days drives logic)
    
    Uses a traffic-light system: ON_TRACK / AT_RISK / VIOLATED.
    Any VIOLATED goal blocks the purchase. AT_RISK goals warn but don't block.
    """

    def __init__(self, budget: float, min_freshness: int, delivery_by: str):
        self.budget = budget
        self.min_freshness = min_freshness
        self.delivery_by = delivery_by
        # Running total: accumulates across multiple purchases in ONE session.
        # This prevents the system from blowing the budget by buying 3 times.
        self.spent = 0.0

    def check(self, price: float, freshness: int, delivery_days: int) -> Dict[str, str]:
        """Check all goals. Returns {goal_name: ON_TRACK|AT_RISK|VIOLATED}.
        
        Budget logic:
          spent + price > budget       → VIOLATED (over budget, hard stop)
          spent + price > budget * 0.8 → AT_RISK  (within 80-100%, warn early)
          otherwise                    → ON_TRACK
          The 80% threshold gives early warning before hitting the wall.
        
        Freshness logic:
          below minimum     → VIOLATED (unacceptable quality)
          exactly minimum   → AT_RISK  (barely acceptable, risky)
          above minimum     → ON_TRACK
        
        Delivery logic (simplified for mock):
          > 3 days → VIOLATED
          > 2 days → AT_RISK
          <= 2 days → ON_TRACK
          In production, you'd compare against the actual deadline date.
        """
        status = {}

        # Budget check
        if self.spent + price > self.budget:
            status["budget"] = "VIOLATED"
        elif self.spent + price > self.budget * 0.8:
            status["budget"] = "AT_RISK"
        else:
            status["budget"] = "ON_TRACK"

        # Freshness check
        if freshness < self.min_freshness:
            status["freshness"] = "VIOLATED"
        elif freshness == self.min_freshness:
            status["freshness"] = "AT_RISK"
        else:
            status["freshness"] = "ON_TRACK"

        # Delivery check
        if delivery_days > 3:
            status["delivery"] = "VIOLATED"
        elif delivery_days > 2:
            status["delivery"] = "AT_RISK"
        else:
            status["delivery"] = "ON_TRACK"

        return status

    def report(self, price: float, freshness: int, delivery_days: int) -> str:
        """Pretty-print version of check() with emoji status icons.
        
        Maps each status to a visual indicator:
          ON_TRACK → ✅    AT_RISK → ⚠️    VIOLATED → ❌
        
        Displayed during negotiation output so the human can see at a glance
        whether this deal is safe to approve before hitting the approval gate.
        """
        status = self.check(price, freshness, delivery_days)
        lines = [f"Goal Status (budget: ${self.budget}, min freshness: {self.min_freshness}, deliver by: {self.delivery_by}):"]
        for goal, state in status.items():
            icon = {"ON_TRACK": "✅", "AT_RISK": "⚠️", "VIOLATED": "❌"}[state]
            lines.append(f"  {icon} {goal}: {state}")
        return "\n".join(lines)


print("GoalTracker ready")

GoalTracker ready


In [6]:
# ── Exception Handling & Recovery ─────────────────────
# Pattern: Exception Handling
# Purpose: The system's incident log + recovery playbook. When something
#          goes wrong (vendor offline, stock depleted, price spike),
#          ExceptionHandler records what happened and tells the orchestrator
#          what to try next. It says WHAT to do; the orchestrator decides HOW.

class ExceptionHandler:
    """Tracks exceptions and provides recovery strategies.
    
    Two responsibilities:
      1. Log every failure with timestamp and context (audit trail)
      2. Map failure types to recovery strategies (playbook)
    
    The recovery strategies are STATIC strings, not actions. This is
    intentional separation of concerns — the handler diagnoses, the
    orchestrator executes the recovery. In production, this could be
    a more sophisticated decision tree or a separate "recovery planner" agent.
    """

    # Static recovery playbook: maps exception types → what to do next.
    # No LLM involved — deterministic, predictable, auditable.
    RECOVERY_STRATEGIES = {
        "OUT_OF_STOCK":     "Try next-best vendor from learning insights",
        "PRICE_SPIKE":      "Negotiate harder or switch to similar fish type",
        "SHIPPING_DELAY":   "Find vendor with local pickup or faster shipping",
        "VENDOR_OFFLINE":   "Skip vendor, use backup from parallel check",
        "QUALITY_REJECTED": "Inspector rejected — try different vendor for same fish",
    }

    def __init__(self):
        # Append-only exception log — every failure gets timestamped.
        # Useful for post-mortem: "Why did the system pick Atlantic Fishery
        # over Wild Salmon Co?" → Check the exception log.
        self.log: List[Dict] = []

    def record(self, exc_type: str, vendor: str, detail: str) -> str:
        """Log an exception and return the suggested recovery strategy.
        
        Called by the orchestrator when a vendor fails at any stage
        (discovery, negotiation, quality check). Returns a human-readable
        string so the orchestrator can print it in the pipeline output.
        
        Unknown exception types get 'Escalate to human' — the safe default.
        Better to ask a human than to guess wrong on an unknown failure.
        """
        entry = {
            "type": exc_type, "vendor": vendor,
            "detail": detail, "timestamp": datetime.now().isoformat(),
            "recovery": self.RECOVERY_STRATEGIES.get(exc_type, "Escalate to human"),
        }
        self.log.append(entry)
        return f"Exception logged: {exc_type} at {vendor}. Recovery: {entry['recovery']}"

    def get_recovery(self, exc_type: str) -> str:
        """Look up recovery strategy without logging.
        
        Used when the orchestrator wants the strategy but has already
        logged the exception via record() — avoids double-logging.
        """
        return self.RECOVERY_STRATEGIES.get(exc_type, "Escalate to human")

    def summary(self) -> str:
        """Human-readable dump of all exceptions.
        
        Shows total count + each event's type, vendor, and detail.
        Displayed in 'Final System State' and when all vendors fail
        so the user can see exactly what went wrong and where.
        """
        if not self.log:
            return "No exceptions recorded."
        lines = [f"Exception log: {len(self.log)} events"]
        for e in self.log:
            lines.append(f"  [{e['type']}] {e['vendor']}: {e['detail']}")
        return "\n".join(lines)


exceptions = ExceptionHandler()
print("ExceptionHandler ready")

ExceptionHandler ready


In [7]:
# ── Agent-to-Agent (A2A) Negotiation Protocol ────────
# Pattern: Agent-to-Agent Communication (A2A)
# Purpose: A STRUCTURED message format for buyer <-> vendor negotiation.
#          Instead of free-text LLM chatter, every negotiation move is a
#          typed message with a protocol version, round number, and specific
#          fields. This makes negotiations auditable, reproducible, parseable.
#
# This is a PROTOCOL, not a conversation. Each message has a fixed schema.
# The vendor's MCP server doesn't use an LLM — it applies deterministic
# business rules (margin check, price midpoint). The protocol makes
# LLM-to-deterministic-code negotiation structured and predictable.

class NegotiationProtocol:
    """Structured protocol for inter-agent price negotiation.
    
    Four message types form the complete negotiation lifecycle:
      OFFER         → Buyer proposes a price (opening move)
      COUNTER_OFFER → Vendor responds with a different price
      ACCEPTED      → Either party locks in the deal (binding)
      REJECTED      → Either party walks away (terminal)
    
    Every message carries:
      - protocol version ("pike_place_a2a_v1") for backward compatibility
      - round number for ordering and audit
      - from/to fields for multi-party traceability
    """

    def __init__(self):
        # Ordered log of ALL negotiation messages in this session.
        # The trace() method turns this into a readable history.
        # In production, you'd persist this for compliance and dispute resolution.
        self.rounds: List[Dict] = []

    def create_offer(self, buyer: str, vendor: str,
                     fish_type: str, qty: float, offer_price: float) -> Dict:
        """Buyer's opening move — propose a price for a specific fish + quantity.
        
        The orchestrator typically starts at 85% of max_price (15% below)
        to leave room for negotiation. The round number auto-increments.
        
        Protocol version 'pike_place_a2a_v1' enables future evolution —
        if we add new fields later, old servers can still parse v1 messages.
        """
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "OFFER",
            "from": buyer, "to": vendor,
            "fish_type": fish_type, "qty_lbs": qty,
            "offer_price_per_lb": offer_price,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def create_counter(self, vendor: str, buyer: str,
                       counter_price: float, available_qty: float) -> Dict:
        """Vendor's response when the offer is too low but negotiable.
        
        Contains the vendor's counter price AND how much they can supply.
        The available_qty might be less than requested (partial fill).
        
        Note: no fish_type here — it's inherited from the original OFFER.
        The buyer reads counter_price and decides: accept, counter again, or walk.
        """
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "COUNTER_OFFER",
            "from": vendor, "to": buyer,
            "counter_price_per_lb": counter_price,
            "available_qty": available_qty,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def accept(self, who: str, final_price: float, qty: float) -> Dict:
        """Either party locks in the deal — this IS the contract.
        
        'who' can be buyer or vendor:
          - Vendor accepts if offer >= list price (instant accept)
          - Buyer accepts if counter <= their max price
        
        Once ACCEPTED is in the trace, the deal is binding. The orchestrator
        proceeds to goal checking and human approval.
        """
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "ACCEPTED",
            "from": who,
            "final_price_per_lb": final_price,
            "final_qty": qty,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def reject(self, who: str, reason: str) -> Dict:
        """Either party walks away — terminal message.
        
        Once REJECTED, the orchestrator moves to the next vendor.
        The reason string gets logged by ExceptionHandler for analysis.
        Common reasons: 'below our minimum', 'cannot agree within budget'.
        """
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "REJECTED",
            "from": who, "reason": reason,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def trace(self) -> str:
        """Full negotiation history — the audit trail.
        
        Prints every round with type-specific details:
          OFFER         → shows qty, fish type, proposed price
          COUNTER_OFFER → shows counter price and available qty
          ACCEPTED      → shows final deal terms (the contract)
          REJECTED      → shows reason (for post-mortem analysis)
        
        In production, you'd persist this for compliance, dispute
        resolution, and feeding back into the LearningEngine.
        """
        lines = [f"Negotiation trace ({len(self.rounds)} rounds):"]
        for r in self.rounds:
            lines.append(f"  Round {r['round']}: {r['type']} from {r['from']}")
            if r['type'] == 'OFFER':
                lines.append(f"    {r['qty_lbs']}lb {r['fish_type']} @ ${r['offer_price_per_lb']}/lb")
            elif r['type'] == 'COUNTER_OFFER':
                lines.append(f"    Counter: ${r['counter_price_per_lb']}/lb, {r['available_qty']}lb available")
            elif r['type'] == 'ACCEPTED':
                lines.append(f"    Deal: {r['final_qty']}lb @ ${r['final_price_per_lb']}/lb")
            elif r['type'] == 'REJECTED':
                lines.append(f"    Reason: {r['reason']}")
        return "\n".join(lines)


print("NegotiationProtocol ready")
print("\nAll infrastructure classes initialized:")
print("  PurchaseMemory, LearningEngine, GoalTracker, ExceptionHandler, NegotiationProtocol")

NegotiationProtocol ready

All infrastructure classes initialized:
  PurchaseMemory, LearningEngine, GoalTracker, ExceptionHandler, NegotiationProtocol


In [8]:
# ── Instantiate infrastructure for this session ──────────
# These global instances persist across all Part 4 phases.
# In Part 5, PurchaseMemory + LearningEngine move to the server.

memory = PurchaseMemory()
learning = LearningEngine(memory)
exceptions = ExceptionHandler()

print("Infrastructure ready:")
print(f"  PurchaseMemory: {len(memory.history)} purchases")
print(f"  LearningEngine: linked to memory")
print(f"  ExceptionHandler: {len(exceptions.log)} events")
print(f"  GoalTracker: created per-scenario (budget varies)")
print(f"  NegotiationProtocol: created per-negotiation")

Infrastructure ready:
  PurchaseMemory: 0 purchases
  LearningEngine: linked to memory
  ExceptionHandler: 0 events
  GoalTracker: created per-scenario (budget varies)
  NegotiationProtocol: created per-negotiation


### How These Classes Migrate to the Server

When we move from mock (Part 4) to real MCP (Part 5), here's what changes:

| Class | Part 4 (Mock) | Part 5 (Real MCP) | Why |
|-------|---------------|-------------------|-----|
| **PurchaseMemory** | In-process Python class | `SqliteSessionService` on server | Server owns persistence — survives restarts |
| **LearningEngine** | In-process, reads from PurchaseMemory | `run_learning_inference` MCP tool | Learning logic moves server-side with the data |
| **GoalTracker** | Client-side | **Stays on client** | Orchestration concern — client decides go/no-go |
| **ExceptionHandler** | Client-side | **Stays on client** | Client handles MCP errors and retry logic |
| **NegotiationProtocol** | Client-side | **Moves to server** | Server owns the negotiation state machine |

The principle: **data and business logic go to the server, orchestration and display stay on the client.**

---

## Part 4: Mock MCP — Training Wheels

Here we simulate MCP's tool interface **without** network or protocol complexity. You'll see every pattern working clearly — every tool call, every agent reasoning step, every negotiation round.

In Part 5, we upgrade to real MCP with FastMCP and SqliteSessionService. The patterns stay the same; only the plumbing changes.

**Why mock first?** The same reason flight simulators exist — you learn the mechanics without the risk. Mock MCP lets you see the 16 patterns clearly. Real MCP adds protocol noise that obscures the patterns on first read.

In [9]:
# ── MCP Vendor Server Simulator ──────────────────────
# Pattern: MCP Server (Model Context Protocol)
# Purpose: In production, each vendor (Wild Salmon Co, Tuna King, etc.)
#          would host their OWN MCP server — a real microservice that
#          agents discover and call via stdio/SSE/HTTP.
#          Here we simulate that with a Python class so the pipeline
#          works identically without network dependencies.
#
# The ADK FunctionTool layer wraps these servers so agents see them
# as normal tools — they don't know (or care) whether the tool calls
# a local class or a remote MCP endpoint.

class MockMCPVendorServer:
    """Simulates an MCP tool server for a Pike Place vendor.
    
    Each instance represents ONE vendor's server with:
      - vendor_name: identity (e.g., "Wild Salmon Co")
      - data:        that vendor's inventory, prices, freshness, cost basis
      - fault:       optional fault injection for testing exception handling
    
    Two MCP operations exposed:
      get_inventory() → what's available and at what price
      negotiate()     → deterministic price negotiation logic
    
    Fault injection allows testing without changing the pipeline code:
      "OUT_OF_STOCK"   → vendor reports 0 inventory
      "PRICE_SPIKE"    → prices jump 50%
      "VENDOR_OFFLINE" → server throws ConnectionError
    """

    def __init__(self, vendor_name: str, vendor_data: Dict,
                 fault_injection: Optional[str] = None):
        self.vendor_name = vendor_name
        self.data = vendor_data
        self.fault = fault_injection  # None = healthy, or "OUT_OF_STOCK" / "PRICE_SPIKE" / "VENDOR_OFFLINE"

    def get_inventory(self, fish_type: str) -> Dict:
        """MCP operation: get_inventory — check what this vendor has.
        
        Returns a dict with: vendor, fish_type, available_lbs, price_per_lb,
        freshness_score, reliability, and status.
        
        Status values:
          "AVAILABLE"    → vendor has this fish in stock
          "OUT_OF_STOCK" → vendor carries this fish but has none right now
          "NOT_CARRIED"  → vendor doesn't sell this fish at all
        
        Fault behavior:
          VENDOR_OFFLINE → raises ConnectionError (caught by ExceptionHandler)
          OUT_OF_STOCK   → returns 0 lbs even if inventory exists
          PRICE_SPIKE    → multiplies price by 1.5x (simulates market volatility)
        """
        # Fault: vendor server is down entirely
        if self.fault == "VENDOR_OFFLINE":
            raise ConnectionError(f"{self.vendor_name} is offline")

        # Fault: vendor shows 0 stock for items they normally carry
        if self.fault == "OUT_OF_STOCK" and fish_type in self.data["inventory"]:
            return {"vendor": self.vendor_name, "fish_type": fish_type,
                    "available_lbs": 0, "status": "OUT_OF_STOCK"}

        # Normal: fish not in this vendor's catalog at all
        if fish_type not in self.data["inventory"]:
            return {"vendor": self.vendor_name, "fish_type": fish_type,
                    "available_lbs": 0, "status": "NOT_CARRIED"}

        price = self.data["price_per_lb"][fish_type]
        # Fault: price spike — 50% increase (e.g., $28 → $42)
        if self.fault == "PRICE_SPIKE":
            price = int(price * 1.5)

        return {
            "vendor": self.vendor_name,
            "fish_type": fish_type,
            "available_lbs": self.data["inventory"][fish_type],
            "price_per_lb": price,
            "freshness_score": self.data["freshness_score"][fish_type],
            "reliability": self.data["reliability"],
            "status": "AVAILABLE",
        }

    def negotiate(self, fish_type: str, qty: float, offer_price: float) -> Dict:
        """MCP operation: negotiate_price — deterministic business rules.
        
        NO LLM involved here. The vendor applies simple margin math:
        
        1. Calculate minimum acceptable price:
           min_price = cost_basis * (1 + min_margin)
           e.g., cost $18, margin 20% → min_price = $21.60
        
        2. Decision tree:
           offer >= list_price  → INSTANT ACCEPT at list price
                                  (vendor won't charge MORE than listed)
           offer >= min_price   → COUNTER at midpoint between offer and list
                                  (vendor meets buyer halfway)
           offer < min_price    → REJECT with reason + counter at min_price
                                  (below cost + margin, can't do it)
        
        The midpoint counter is the key negotiation mechanic:
          List = $28, Offer = $24, Counter = ($24 + $28) / 2 = $26
          This creates a natural convergence toward agreement.
        """
        if fish_type not in self.data["cost_basis"]:
            return {"accepted": False, "reason": "Fish not carried"}

        cost = self.data["cost_basis"][fish_type]
        min_price = cost * (1 + self.data["min_margin"])

        if offer_price >= self.data["price_per_lb"][fish_type]:
            # Buyer offered list price or higher — instant accept at list price
            # (vendor doesn't gouge; they accept at their listed rate)
            return {"accepted": True, "final_price": self.data["price_per_lb"][fish_type], "qty": qty}
        elif offer_price >= min_price:
            # Offer is above minimum margin — counter at midpoint
            # This is where negotiation happens: vendor meets buyer halfway
            counter = round((offer_price + self.data["price_per_lb"][fish_type]) / 2, 2)
            return {"accepted": False, "counter_price": counter, "available_qty": qty}
        else:
            # Below minimum — hard reject, no counter-offer
            # The vendor walks away. Orchestrator must try next vendor.
            # BUG FIX: previously included counter_price here, which meant
            # the orchestrator's `result.get("counter_price")` check treated
            # this as a counter-offer instead of a rejection. Removing
            # counter_price ensures the orchestrator falls through to the
            # reject handler correctly.
            return {"accepted": False, "reason": f"Offer ${offer_price}/lb is below our minimum of ${min_price:.2f}/lb"}


# Create MCP servers for all vendors (no faults by default)
# In production: these would be real MCP server connections via McpToolset
mcp_servers = {
    name: MockMCPVendorServer(name, data)
    for name, data in VENDORS.items()
}
print(f"MCP servers created for {len(mcp_servers)} vendors")

MCP servers created for 5 vendors


In [10]:
# ── Tool Call Logger ───────────────────────────────────
# Tracks every tool invocation: which tool, when, what args.
# Lets you see whether the LLM called tools in parallel or sequentially.
# If two calls have the same timestamp (within ~0.01s), they ran in parallel.

import time as _time

class ToolLogger:
    """Lightweight logger that tracks tool call sequence and timing.
    
    Usage: check tool_logger.log after running a pipeline to see
    the full call sequence with timestamps. Parallel calls will
    have near-identical timestamps (within ~10ms).
    """
    def __init__(self):
        self.log = []           # [(seq, timestamp, tool_name, args_summary)]
        self._seq = 0
        self._start = _time.time()
    
    def call(self, tool_name: str, args: dict) -> int:
        self._seq += 1
        elapsed = round(_time.time() - self._start, 3)
        args_short = ", ".join(f"{k}={repr(v)[:30]}" for k, v in args.items())
        entry = (self._seq, elapsed, tool_name, args_short)
        self.log.append(entry)
        print(f"    🔧 [{self._seq:2d}] +{elapsed:6.3f}s  {tool_name}({args_short})")
        return self._seq
    
    def reset(self):
        self.log.clear()
        self._seq = 0
        self._start = _time.time()
    
    def summary(self):
        """Print the full call log with parallel-detection."""
        print(f"\n{'─'*60}")
        print(f"TOOL CALL LOG ({len(self.log)} calls)")
        print(f"{'─'*60}")
        prev_time = -1
        for seq, elapsed, name, args in self.log:
            # If two calls are within 10ms, they were likely parallel
            parallel = "  ⚡ PARALLEL" if abs(elapsed - prev_time) < 0.01 else ""
            print(f"  [{seq:2d}] +{elapsed:6.3f}s  {name}({args}){parallel}")
            prev_time = elapsed
        print(f"{'─'*60}")

tool_logger = ToolLogger()

# ── ADK Tool Functions (wrap MCP servers) ─────────────
# These are the BRIDGE between LLM agents and the MCP server layer.
# Each function is wrapped in FunctionTool() so ADK agents can call them.
# The agent sees: "I have a tool called query_vendor_catalog"
# Under the hood: it calls MockMCPVendorServer.get_inventory() for each vendor.
#
# In production with real MCP servers, you'd replace these with McpToolset
# connections — the agents wouldn't need to change at all.

def query_vendor_catalog(fish_type: str) -> str:
    """Query ALL Pike Place vendors for a specific fish type.
    
    This is the discovery tool — the BuyerAgent's first call.
    Loops through every vendor's MCP server, collects availability,
    and returns a ranked JSON array sorted by:
      1. Freshness (descending) — quality first
      2. Price (ascending) — cheapest among same quality
    
    Exception handling: if a vendor is offline (VENDOR_OFFLINE fault),
    the ConnectionError is caught and logged to ExceptionHandler.
    The agent never sees the error — it just gets fewer results.
    
    Returns JSON array of available vendors, or a "no vendors" message.
    """
    tool_logger.call("query_vendor_catalog", {"fish_type": fish_type})
    results = []
    for name, server in mcp_servers.items():
        try:
            inv = server.get_inventory(fish_type)
            if inv["status"] == "AVAILABLE":
                results.append(inv)
            elif inv["status"] == "OUT_OF_STOCK":
                results.append(inv)
        except ConnectionError as e:
            # Vendor server is down — log it but don't crash the pipeline
            exceptions.record("VENDOR_OFFLINE", name, str(e))
    if not results:
        return f"No vendors carry {fish_type} at Pike Place Market."
    # Sort: best freshness first, then cheapest within same freshness
    available = [r for r in results if r["status"] == "AVAILABLE"]
    available.sort(key=lambda x: (-x["freshness_score"], x["price_per_lb"]))
    return json.dumps(available, indent=2)


def check_freshness(vendor_name: str, fish_type: str) -> str:
    """Check detailed freshness info for a specific vendor + fish combo.
    
    Called by the QualityInspector agent to verify that a vendor's fish
    meets the customer's quality requirements.
    
    Maps freshness scores to quality tiers:
      9-10 → EXCELLENT (sashimi-grade)
      7-8  → GOOD (cooking-grade)
      5-6  → FAIR (not recommended for raw)
      1-4  → POOR (avoid)
    
    Also reports sashimi suitability (freshness >= 8 required).
    Returns a human-readable string the inspector agent can reason about.
    """
    tool_logger.call("check_freshness", {"vendor": vendor_name, "fish": fish_type})
    server = mcp_servers.get(vendor_name)
    if not server:
        return f"Unknown vendor: {vendor_name}"
    try:
        inv = server.get_inventory(fish_type)
        if inv["status"] != "AVAILABLE":
            return f"{vendor_name} does not have {fish_type} ({inv['status']})"
        freshness = inv["freshness_score"]
        quality = "EXCELLENT" if freshness >= 9 else "GOOD" if freshness >= 7 else "FAIR" if freshness >= 5 else "POOR"
        return (f"{vendor_name} {fish_type}: freshness {freshness}/10 ({quality}). "
                f"Suitable for sashimi: {'Yes' if freshness >= 8 else 'No'}")
    except ConnectionError:
        return f"{vendor_name} is offline"


def estimate_shipping(vendor_name: str, qty_lbs: float) -> str:
    """Estimate shipping cost and delivery time from a vendor.
    
    Mock implementation:
      Cost = $5 base + $1 per pound (overnight cold pack)
      Delivery = 1 day for Wild Salmon Co and Atlantic Fishery (local),
                 2 days for all others (slightly further)
    
    Returns JSON with: vendor, shipping_cost, delivery_days, method.
    Used by both BuyerAgent (to calculate total cost) and GoalTracker
    (to check delivery deadline constraints).
    """
    tool_logger.call("estimate_shipping", {"vendor": vendor_name, "qty": qty_lbs})
    cost = 5 + qty_lbs * 1.0
    days = 1 if vendor_name in ["Wild Salmon Co", "Atlantic Fishery"] else 2
    return json.dumps({
        "vendor": vendor_name, "shipping_cost": cost,
        "delivery_days": days, "method": "overnight cold pack",
    })


def negotiate_price(vendor_name: str, fish_type: str,
                    qty_lbs: float, offer_price_per_lb: float) -> str:
    """Propose a price to a vendor — triggers the A2A negotiation logic.
    
    This is the tool interface to MockMCPVendorServer.negotiate().
    The agent provides an offer price; the vendor responds with:
      - accepted: True + final_price (deal done)
      - accepted: False + counter_price (keep negotiating)
      - accepted: False + reason (rejected, walk away)
    
    Returns JSON so the NegotiatorAgent or orchestrator can parse
    the response and decide next move (accept counter, counter again, 
    or try next vendor).
    """
    tool_logger.call("negotiate_price", {"vendor": vendor_name, "offer": offer_price_per_lb})
    server = mcp_servers.get(vendor_name)
    if not server:
        return f"Unknown vendor: {vendor_name}"
    result = server.negotiate(fish_type, qty_lbs, offer_price_per_lb)
    return json.dumps(result)


def get_learning_insights(fish_type: str) -> str:
    """Get insights from past purchases about this fish type.
    
    Thin wrapper around LearningEngine.get_insights().
    Called by BuyerAgent BEFORE searching for vendors — this is how
    the system uses its memory to make smarter decisions.
    
    Returns either:
      "No purchase history for X" → first-time buy, no guidance
      "Learning insights: best vendor, avg price, ..." → use this data
    
    The BuyerAgent reads this and adjusts its vendor preference accordingly.
    """
    tool_logger.call("get_learning_insights", {"fish_type": fish_type})
    return learning.get_insights(fish_type)


def record_purchase(vendor_name: str, fish_type: str, qty_lbs: float,
                    price_per_lb: float, freshness_score: int) -> str:
    """Record a completed purchase to memory for future learning.
    
    Called AFTER human approval — this is the final step in the pipeline.
    Feeds PurchaseMemory.add_purchase() which:
      1. Stores the purchase record (for get_history_for queries)
      2. Updates vendor reliability scores (for get_vendor_reliability)
    
    The on_time=True default is because our mock always delivers on time.
    In production, you'd check actual delivery status before recording.
    """
    tool_logger.call("record_purchase", {"vendor": vendor_name, "fish": fish_type, "qty": qty_lbs})
    record = memory.add_purchase(vendor_name, fish_type, qty_lbs,
                                 price_per_lb, freshness_score, on_time=True)
    return f"Purchase recorded: {qty_lbs}lb {fish_type} from {vendor_name} @ ${price_per_lb}/lb"


def check_goals(total_price: float, freshness_score: int,
                delivery_days: int) -> str:
    """Check all purchase goals (budget, freshness, delivery).
    
    Placeholder tool — in the current architecture, GoalTracker.check()
    is called directly by the orchestrator (not via an agent tool call).
    
    This exists so that a future GoalMonitor agent could call it
    independently. The message reminds developers that goal checking
    is handled by the orchestrator's Python code, not by an LLM.
    """
    tool_logger.call("check_goals", {"total": total_price, "freshness": freshness_score})
    return "Goal check requires active GoalTracker — see orchestrator."


print("Tools defined:")
for fn in [query_vendor_catalog, check_freshness, estimate_shipping,
           negotiate_price, get_learning_insights, record_purchase, check_goals]:
    print(f"  {fn.__name__}")

Tools defined:
  query_vendor_catalog
  check_freshness
  estimate_shipping
  negotiate_price
  get_learning_insights
  record_purchase
  check_goals


### Phase 1: Discovery (Routing + Parallelization + Tool Use + Reflection)

The customer says what they want. The system:
1. **BuyerAgent** queries all vendors via tools (Parallelization, Tool Use)
2. **QualityInspector** reviews options against freshness requirements (Reflection)

```
Customer: "5 lbs King Salmon for Saturday dinner, budget $150, sashimi-grade"
         │
    [BuyerAgent] ── queries vendors, checks freshness, estimates shipping
         │
    [QualityInspector] ── rejects freshness < 8 ── returns approved list
```

In [11]:
# ── Phase 1 Agents ──────────────────────────────────

buyer_agent = LlmAgent(
    name="BuyerAgent",
    model=MODEL,
    instruction="""
You are a fish buyer at Pike Place Market. Given a customer request,
use query_vendor_catalog to find all vendors with the requested fish.

Then for the top options, use check_freshness to verify quality.
Also use estimate_shipping to check delivery feasibility.

Before searching, call get_learning_insights to check if we have
past purchase data that can guide vendor selection.

Return a ranked list of options with:
- Vendor name, price/lb, freshness score, shipping cost, delivery days
- Recommendation: which vendor is best and why
""",
    tools=[
        FunctionTool(query_vendor_catalog),
        FunctionTool(check_freshness),
        FunctionTool(estimate_shipping),
        FunctionTool(get_learning_insights),
    ],
    generate_content_config=RETRY_CONFIG,
)

inspector_agent = LlmAgent(
    name="QualityInspector",
    model=MODEL,
    instruction="""
You are a quality inspector for Pike Place Market purchases.
Review the buyer's vendor recommendations and check freshness
for each option using the check_freshness tool.

REJECT any option where:
- Freshness score is below 7 (for cooking) or below 8 (if customer wants sashimi/raw)
- Vendor reliability is below 85%

Return only APPROVED options with your quality assessment.
If no options pass inspection, say so clearly.
""",
    tools=[FunctionTool(check_freshness)],
    generate_content_config=RETRY_CONFIG,
)

print("Phase 1 agents ready: BuyerAgent, QualityInspector")

Phase 1 agents ready: BuyerAgent, QualityInspector


In [12]:
# ── Phase 1: Run Discovery Pipeline ──────────────────

async def run_discovery(customer_request: str) -> str:
    """Phase 1: Discover and quality-check vendor options."""
    print(f"{'='*60}")
    print(f"CUSTOMER: {customer_request}")
    tool_logger.reset()  # Reset tool log for this pipeline run
    print(f"{'='*60}")

    session_service = InMemorySessionService()

    # Step 1: Buyer discovers options
    print("\n[BUYER] Searching vendors...")
    buyer_runner = Runner(agent=buyer_agent, app_name="pike_place",
                          session_service=session_service)
    session = await session_service.create_session(
        app_name="pike_place", user_id="customer")

    content = types.Content(role="user",
        parts=[types.Part.from_text(text=customer_request)])

    buyer_response = ""
    tool_calls = 0
    # ── ADK Event Stream ─────────────────────────────────────────
    # runner.run_async() returns an async generator of Event objects.
    # Each Event is one step in the agent's turn — streamed in real time:
    #
    #   Event 1: function_call      → LLM decided to call a tool (name + args)
    #   Event 2: function_response   → Tool returned its result
    #   Event 3: function_call      → LLM calls another tool
    #   Event 4: function_response   → That tool's result
    #   Event 5: text               → LLM synthesizes a final answer
    #   Event 6: is_final_response  → Agent is done (break the loop)
    #
    # Each event.content.parts[] can contain:
    #   part.function_call     → tool invocation (name, args)
    #   part.function_response → tool result (response dict)
    #   part.text              → LLM reasoning or final answer
    #
    # is_final_response() is the exit signal — the agent has nothing
    # more to do. The dedup check (part.text not in response) prevents
    # double-counting text that was already streamed in earlier events.
    # ──────────────────────────────────────────────────────────────
    async for event in buyer_runner.run_async(
        user_id="customer", session_id=session.id, new_message=content):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    tool_calls += 1
                    print(f"  [{tool_calls}] Tool: {part.function_call.name}({dict(part.function_call.args)})")
                elif hasattr(part, "function_response") and part.function_response:
                    result = str(part.function_response.response)[:150]
                    print(f"      Result: {result}...")
                elif hasattr(part, "text") and part.text and part.text.strip():
                    buyer_response += part.text
        if event.is_final_response():
            # Also try to get text from the final event's content
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text and part.text.strip():
                        if part.text not in buyer_response:
                            buyer_response += part.text
            break

    print(f"\n[BUYER] Found options ({tool_calls} tool calls)")
    if buyer_response:
        print(f"\n[BUYER RESPONSE]\n{buyer_response[:500]}")

    # Step 2: Inspector quality-checks
    print("\n[INSPECTOR] Reviewing quality...")
    inspector_runner = Runner(agent=inspector_agent, app_name="pike_place",
                              session_service=session_service)
    inspect_session = await session_service.create_session(
        app_name="pike_place", user_id="inspector")

    inspect_msg = types.Content(role="user",
        parts=[types.Part.from_text(
            text=f"Review these vendor options for the customer who wants: {customer_request}\n\n"
                 f"Buyer's findings:\n{buyer_response}")])

    inspector_response = ""
    # ── Same event stream pattern as buyer above ──────────────
    async for event in inspector_runner.run_async(
        user_id="inspector", session_id=inspect_session.id, new_message=inspect_msg):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    print(f"  Inspector checking: {part.function_call.name}({dict(part.function_call.args)})")
                elif hasattr(part, "text") and part.text and part.text.strip():
                    inspector_response += part.text
        if event.is_final_response():
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text and part.text.strip():
                        if part.text not in inspector_response:
                            inspector_response += part.text
            break

    print(f"\n[INSPECTOR] Quality review complete")
    print(f"\n{'='*60}")
    print(f"APPROVED OPTIONS:\n{inspector_response[:500] if inspector_response else '(No text response — inspector may have only used tool calls)'}")
    print(f"{'='*60}")
    tool_logger.summary()  # Show full tool call sequence with parallel detection
    return inspector_response


print("Discovery pipeline ready: run_discovery()")

Discovery pipeline ready: run_discovery()


In [13]:
# ── Run Phase 1 ─────────────────────────────────────

discovery_result = await run_discovery(
    "I need 5 lbs of fresh King Salmon for a dinner party this Saturday. "
    "Budget is $150. It needs to be super fresh — I'm serving it as sashimi."
)

CUSTOMER: I need 5 lbs of fresh King Salmon for a dinner party this Saturday. Budget is $150. It needs to be super fresh — I'm serving it as sashimi.

[BUYER] Searching vendors...


  [1] Tool: get_learning_insights({'fish_type': 'King Salmon'})
    🔧 [ 1] + 1.027s  get_learning_insights(fish_type='King Salmon')
      Result: {'result': 'No purchase history for King Salmon. This is our first time buying it.'}...
  [2] Tool: query_vendor_catalog({'fish_type': 'King Salmon'})
    🔧 [ 2] + 1.739s  query_vendor_catalog(fish_type='King Salmon')
      Result: {'result': '[\n  {\n    "vendor": "Wild Salmon Co",\n    "fish_type": "King Salmon",\n    "available_lbs": 25,\n    "price_per_lb": 28,\n    "freshnes...
  [3] Tool: check_freshness({'fish_type': 'King Salmon', 'vendor_name': 'Wild Salmon Co'})
    🔧 [ 3] + 2.519s  check_freshness(vendor='Wild Salmon Co', fish='King Salmon')
      Result: {'result': 'Wild Salmon Co King Salmon: freshness 9/10 (EXCELLENT). Suitable for sashimi: Yes'}...
  [4] Tool: estimate_shipping({'qty_lbs': 5, 'vendor_name': 'Wild Salmon Co'})
    🔧 [ 4] + 3.379s  estimate_shipping(vendor='Wild Salmon Co', qty=5)
      Result: {'result': '{"vend

### Phase 2: Negotiation & Human Approval (A2A + Goals + HITL)

Now the system:
1. **NegotiationProtocol** structures the offer/counter-offer exchange
2. **GoalTracker** checks budget, freshness, and delivery constraints
3. **Human Approval Gate** — customer confirms before purchase

```
[Negotiator] ──► OFFER: 5lb King Salmon @ $23.80/lb
     │
[Vendor]     ──► COUNTER: $25.90/lb
     │
[Negotiator] ──► ACCEPT: $25.90/lb × 5lb = $129.50
     │
[GoalTracker]──► Budget: AT_RISK, Freshness: ON_TRACK
     │
[Human]      ──► "Approve $139.50 total?" → YES
```

In [14]:
# ── Phase 2: Negotiation + Goals + Human Approval ───────

async def run_negotiation_and_approval(
    vendor_name: str, fish_type: str, qty: float,
    max_price_per_lb: float, goals: GoalTracker,
    auto_approve: bool = True  # set False for real human input
) -> Optional[Dict]:
    """Phase 2: Negotiate price, check goals, get human approval."""
    protocol = NegotiationProtocol()

    print(f"\n{'='*60}")
    print(f"NEGOTIATION: {qty}lb {fish_type} from {vendor_name}")
    print(f"Max price: ${max_price_per_lb}/lb (budget: ${goals.budget})")
    print(f"{'='*60}")

    # ── Round 1: Buyer offers below list price ───────────
    offer_price = max_price_per_lb * 0.85  # start 15% below max
    offer = protocol.create_offer("BuyerAgent", vendor_name,
                                   fish_type, qty, round(offer_price, 2))
    print(f"\n[BUYER] Offer: ${offer_price:.2f}/lb")

    # ── Vendor responds via MCP ───────────────────────
    server = mcp_servers[vendor_name]
    result = server.negotiate(fish_type, qty, offer_price)

    if result.get("accepted"):
        final_price = result["final_price"]
        protocol.accept(vendor_name, final_price, qty)
        print(f"[VENDOR] Accepted at ${final_price}/lb!")
    elif result.get("counter_price"):
        counter = result["counter_price"]
        protocol.create_counter(vendor_name, "BuyerAgent", counter, qty)
        print(f"[VENDOR] Counter-offer: ${counter}/lb")

        # ── Round 2: Buyer meets in the middle ──────────
        if counter <= max_price_per_lb:
            final_price = counter
            protocol.accept("BuyerAgent", final_price, qty)
            print(f"[BUYER] Accepted counter at ${final_price}/lb")
        else:
            # Try one more: split the difference
            split = round((offer_price + counter) / 2, 2)
            offer2 = protocol.create_offer("BuyerAgent", vendor_name,
                                            fish_type, qty, split)
            print(f"[BUYER] Counter-counter: ${split}/lb")
            result2 = server.negotiate(fish_type, qty, split)
            if result2.get("accepted") or (result2.get("counter_price") and result2["counter_price"] <= max_price_per_lb):
                final_price = result2.get("final_price", result2.get("counter_price"))
                protocol.accept("BuyerAgent", final_price, qty)
                print(f"[DEAL] Agreed at ${final_price}/lb")
            else:
                protocol.reject("BuyerAgent", "Cannot agree on price within budget")
                print(f"[NO DEAL] Cannot agree within budget")
                print(f"\n{protocol.trace()}")
                return None
    else:
        protocol.reject(vendor_name, result.get("reason", "Unknown"))
        print(f"[VENDOR] Rejected: {result.get('reason')}")
        print(f"\n{protocol.trace()}")
        return None

    total_cost = final_price * qty
    shipping = json.loads(estimate_shipping(vendor_name, qty))
    total_with_shipping = total_cost + shipping["shipping_cost"]

    # ── Goal Check ──────────────────────────────────
    freshness = VENDORS[vendor_name]["freshness_score"].get(fish_type, 5)
    print(f"\n{goals.report(total_with_shipping, freshness, shipping['delivery_days'])}")

    goal_status = goals.check(total_with_shipping, freshness, shipping["delivery_days"])
    if "VIOLATED" in goal_status.values():
        violated = [g for g, s in goal_status.items() if s == "VIOLATED"]
        print(f"\n❌ Goal violation: {', '.join(violated)}")
        return None

    # ── Human Approval Gate ─────────────────────────
    print(f"\n{'='*60}")
    print(f"PURCHASE SUMMARY:")
    print(f"  {qty}lb {fish_type} from {vendor_name}")
    print(f"  Price: ${final_price}/lb × {qty}lb = ${total_cost:.2f}")
    print(f"  Shipping: ${shipping['shipping_cost']:.2f} ({shipping['delivery_days']} days)")
    print(f"  Total: ${total_with_shipping:.2f}")
    print(f"  Freshness: {freshness}/10")
    print(f"{'='*60}")

    if auto_approve:
        print("\n✅ [HUMAN] Auto-approved (set auto_approve=False for manual)")
        approved = True
    else:
        response = input("Approve this purchase? (yes/no): ")
        approved = response.strip().lower() in ["yes", "y"]

    if not approved:
        print("❌ Purchase rejected by customer")
        return None

    # ── Record to Memory ────────────────────────────
    record_purchase(vendor_name, fish_type, qty, final_price, freshness)
    goals.spent += total_with_shipping

    deal = {
        "vendor": vendor_name, "fish_type": fish_type,
        "qty": qty, "price_per_lb": final_price,
        "total": total_with_shipping, "freshness": freshness,
        "delivery_days": shipping["delivery_days"],
    }

    print(f"\n{protocol.trace()}")
    print(f"\n✅ Purchase complete! Recorded to memory for future learning.")
    return deal


print("Negotiation pipeline ready: run_negotiation_and_approval()")

Negotiation pipeline ready: run_negotiation_and_approval()


In [15]:
# ── Run Phase 2: Negotiate with Wild Salmon Co ──────────

goals = GoalTracker(budget=150, min_freshness=8, delivery_by="Saturday")

deal = await run_negotiation_and_approval(
    vendor_name="Wild Salmon Co",
    fish_type="King Salmon",
    qty=5.0,
    max_price_per_lb=28.0,
    goals=goals,
    auto_approve=True,
)


NEGOTIATION: 5.0lb King Salmon from Wild Salmon Co
Max price: $28.0/lb (budget: $150)

[BUYER] Offer: $23.80/lb
[VENDOR] Counter-offer: $25.9/lb
[BUYER] Accepted counter at $25.9/lb
    🔧 [ 8] + 8.001s  estimate_shipping(vendor='Wild Salmon Co', qty=5.0)

Goal Status (budget: $150, min freshness: 8, deliver by: Saturday):
  ⚠️ budget: AT_RISK
  ✅ freshness: ON_TRACK
  ✅ delivery: ON_TRACK

PURCHASE SUMMARY:
  5.0lb King Salmon from Wild Salmon Co
  Price: $25.9/lb × 5.0lb = $129.50
  Shipping: $10.00 (1 days)
  Total: $139.50
  Freshness: 9/10

✅ [HUMAN] Auto-approved (set auto_approve=False for manual)
    🔧 [ 9] + 8.001s  record_purchase(vendor='Wild Salmon Co', fish='King Salmon', qty=5.0)

Negotiation trace (3 rounds):
  Round 1: OFFER from BuyerAgent
    5.0lb King Salmon @ $23.8/lb
  Round 2: COUNTER_OFFER from Wild Salmon Co
    Counter: $25.9/lb, 5.0lb available
  Round 3: ACCEPTED from BuyerAgent
    Deal: 5.0lb @ $25.9/lb

✅ Purchase complete! Recorded to memory for future l

### Phase 3: Exception Handling & Recovery

What happens when things go wrong? We inject faults and watch the system adapt:

| Fault | What Happens | Recovery Strategy |
|-------|-------------|------------------|
| `OUT_OF_STOCK` | Vendor has 0 inventory | Try next-best vendor |
| `PRICE_SPIKE` | Price jumped 50% | Negotiate harder or switch vendor |
| `VENDOR_OFFLINE` | MCP server unreachable | Skip, use backup vendor |

The **ExceptionHandler** logs every failure. The system retries with the next-best option.

In [16]:
# ── Phase 3: Exception Handling with Fault Injection ─────

async def run_with_exceptions(
    fish_type: str, qty: float, max_price: float,
    budget: float, fault_vendor: str, fault_type: str
) -> Optional[Dict]:
    """Run the full pipeline with a fault injected at one vendor."""
    print(f"\n{'='*60}")
    print(f"SCENARIO: {fault_type} at {fault_vendor}")
    print(f"{'='*60}")

    # Inject fault into one vendor's MCP server
    original_server = mcp_servers[fault_vendor]
    mcp_servers[fault_vendor] = MockMCPVendorServer(
        fault_vendor, VENDORS[fault_vendor], fault_injection=fault_type)

    # Discover options (some will fail)
    print(f"\n[BUYER] Querying vendors for {fish_type}...")
    catalog_result = query_vendor_catalog(fish_type)
    options = json.loads(catalog_result) if catalog_result.startswith('[') else []

    if not options:
        print(f"  No available vendors! Exception log:")
        print(f"  {exceptions.summary()}")
        mcp_servers[fault_vendor] = original_server
        return None

    print(f"  Found {len(options)} available vendors:")
    for opt in options:
        print(f"    {opt['vendor']}: ${opt['price_per_lb']}/lb, "
              f"freshness {opt['freshness_score']}/10, {opt['available_lbs']}lb")

    # Try vendors in order until one works
    goals = GoalTracker(budget=budget, min_freshness=7, delivery_by="Saturday")
    deal = None

    for option in options:
        vendor = option["vendor"]
        print(f"\n[TRYING] {vendor}...")

        try:
            deal = await run_negotiation_and_approval(
                vendor_name=vendor, fish_type=fish_type,
                qty=qty, max_price_per_lb=max_price,
                goals=goals, auto_approve=True)

            if deal:
                print(f"\n✅ Success with {vendor}!")
                break
        except Exception as e:
            exc_msg = exceptions.record(type(e).__name__, vendor, str(e))
            print(f"  ❌ {exc_msg}")
            print(f"  Recovery: {exceptions.get_recovery(type(e).__name__)}")

    # Restore original server
    mcp_servers[fault_vendor] = original_server

    if not deal:
        print(f"\n❌ All vendors exhausted. Exception summary:")
        print(exceptions.summary())

    return deal


print("Exception handling pipeline ready")

Exception handling pipeline ready


In [17]:
# ── Scenario: Wild Salmon Co is OUT OF STOCK ───────────
# System should detect this and fall back to Atlantic Fishery

deal = await run_with_exceptions(
    fish_type="King Salmon", qty=5.0, max_price=28.0,
    budget=150.0,
    fault_vendor="Wild Salmon Co", fault_type="OUT_OF_STOCK"
)


SCENARIO: OUT_OF_STOCK at Wild Salmon Co

[BUYER] Querying vendors for King Salmon...
    🔧 [10] + 8.009s  query_vendor_catalog(fish_type='King Salmon')
  Found 1 available vendors:
    Atlantic Fishery: $18/lb, freshness 6/10, 50lb

[TRYING] Atlantic Fishery...

NEGOTIATION: 5.0lb King Salmon from Atlantic Fishery
Max price: $28.0/lb (budget: $150.0)

[BUYER] Offer: $23.80/lb
[VENDOR] Accepted at $18/lb!
    🔧 [11] + 8.009s  estimate_shipping(vendor='Atlantic Fishery', qty=5.0)

Goal Status (budget: $150.0, min freshness: 7, deliver by: Saturday):
  ✅ budget: ON_TRACK
  ❌ freshness: VIOLATED
  ✅ delivery: ON_TRACK

❌ Goal violation: freshness

❌ All vendors exhausted. Exception summary:
No exceptions recorded.


### Phase 4: Learning in Action (Memory + Learning)

Run the pipeline multiple times. Watch the system get smarter:
- **Run 1:** No history — buyer checks all vendors equally
- **Run 2+:** Learning engine recommends best vendor from past experience
- The `get_learning_insights` tool returns accumulated wisdom

In [18]:
# ── Learning Demo: Multiple Purchase Cycles ────────────

# Reset memory for clean demo
memory = PurchaseMemory()
learning = LearningEngine(memory)

print("=" * 60)
print("LEARNING DEMO: 3 purchase cycles for King Salmon")
print("=" * 60)

# Simulate 3 purchases from different vendors
purchases = [
    ("Wild Salmon Co", "King Salmon", 5, 28, 9, True),
    ("Atlantic Fishery", "King Salmon", 3, 18, 6, True),
    ("Wild Salmon Co", "King Salmon", 4, 27, 10, True),
]

for i, (vendor, fish, qty, price, fresh, on_time) in enumerate(purchases, 1):
    print(f"\n--- Purchase #{i} ---")
    memory.add_purchase(vendor, fish, qty, price, fresh, on_time)
    print(f"  Bought {qty}lb {fish} from {vendor} @ ${price}/lb (freshness: {fresh}/10)")
    
    # Check what the system has learned
    insights = learning.get_insights(fish)
    print(f"\n  {insights}")

print(f"\n{'='*60}")
print(f"FINAL MEMORY STATE:")
print(memory.summary())
print(f"\nBest vendor for King Salmon: {learning.best_vendor_for('King Salmon')}")
print(f"Average price paid: ${learning.avg_price('King Salmon'):.2f}/lb")

LEARNING DEMO: 3 purchase cycles for King Salmon

--- Purchase #1 ---
  Bought 5lb King Salmon from Wild Salmon Co @ $28/lb (freshness: 9/10)

  Learning insights for King Salmon:
  Past purchases: 1
  Best vendor: Wild Salmon Co
  Average price: $28.00/lb
  Best freshness seen: 9/10

--- Purchase #2 ---
  Bought 3lb King Salmon from Atlantic Fishery @ $18/lb (freshness: 6/10)

  Learning insights for King Salmon:
  Past purchases: 2
  Best vendor: Atlantic Fishery
  Average price: $23.00/lb
  Best freshness seen: 9/10

--- Purchase #3 ---
  Bought 4lb King Salmon from Wild Salmon Co @ $27/lb (freshness: 10/10)

  Learning insights for King Salmon:
  Past purchases: 3
  Best vendor: Wild Salmon Co
  Average price: $24.33/lb
  Best freshness seen: 10/10

FINAL MEMORY STATE:
Purchase history: 3 transactions
  Wild Salmon Co: 2 purchases, reliability 0.95
  Atlantic Fishery: 1 purchases, reliability 0.60

Best vendor for King Salmon: Wild Salmon Co
Average price paid: $24.33/lb


### Phase 5: Full Pipeline — All 16 Patterns Composed

The `CapstoneOrchestrator` drives the entire pipeline. It's Python code (not an LLM) — same principle as Chapter 8b's Plan-and-Execute pattern.

```
[CapstoneOrchestrator.run()]
  ├─ Phase 1: DISCOVERY (BuyerAgent queries vendors)
  ├─ Phase 2: LEARNING (LearningEngine recommends vendor)
  ├─ Phase 3: NEGOTIATION (NegotiationProtocol + GoalTracker + Approval)
  ├─ Phase 4: RECORDING (PurchaseMemory updates)
  └─ EXCEPTION HANDLING (at any step: log, recover, retry next vendor)
```

In [19]:
# ── Capstone Orchestrator ─────────────────────────────

class CapstoneOrchestrator:
    """Orchestrates the complete Pike Place fish-buying pipeline.
    
    Composes all 14 patterns into one deterministic flow.
    The orchestrator is Python (not an LLM) — same principle as Ch 8b.
    """

    def __init__(self):
        self.session_service = InMemorySessionService()
        self.buyer_runner = Runner(
            agent=buyer_agent, app_name="capstone",
            session_service=self.session_service)

    async def _call_buyer(self, message: str) -> str:
        """Send a message to the buyer agent."""
        session = await self.session_service.create_session(
            app_name="capstone", user_id="orchestrator")
        content = types.Content(role="user",
            parts=[types.Part.from_text(text=message)])
        response = ""
        # ── ADK Event Stream ──────────────────────────────────────
        # Each Event from run_async() is one agent step, streamed live:
        #   part.function_call     → LLM invoked a tool (name + args)
        #   part.function_response → tool returned a result
        #   part.text              → LLM reasoning or final answer
        # is_final_response() = agent is done → break the loop
        # ─────────────────────────────────────────────────────────
        async for event in self.buyer_runner.run_async(
            user_id="orchestrator", session_id=session.id, new_message=content):
            if event.is_final_response() and event.content:
                for part in event.content.parts:
                    if part.text:
                        response += part.text
        return response.strip()

    async def run(self, customer_request: str, budget: float,
                  min_freshness: int = 7, delivery_by: str = "Saturday",
                  fault_vendor: str = None, fault_type: str = None,
                  auto_approve: bool = True) -> Optional[Dict]:
        """Execute the complete fish-buying pipeline."""

        print(f"\n{'#'*60}")
        print(f"# CAPSTONE PIPELINE")
        print(f"# Request: {customer_request}")
        print(f"# Budget: ${budget}, Min Freshness: {min_freshness}/10")
        print(f"{'#'*60}")

        tool_logger.reset()  # Reset tool log for this pipeline run

        goals = GoalTracker(budget=budget, min_freshness=min_freshness,
                            delivery_by=delivery_by)

        # Inject fault if requested
        original_server = None
        if fault_vendor and fault_type:
            original_server = mcp_servers[fault_vendor]
            mcp_servers[fault_vendor] = MockMCPVendorServer(
                fault_vendor, VENDORS[fault_vendor], fault_injection=fault_type)
            print(f"\n⚠️  Fault injected: {fault_type} at {fault_vendor}")

        try:
            # ── Phase 1: Discovery via LLM Buyer Agent ───────
            print(f"\n┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)")
            buyer_result = await self._call_buyer(customer_request)
            print(f"│  Buyer found options")

            # ── Phase 2: Parse available vendors from catalog ──
            # (In parallel pipeline, we'd use ParallelAgent here)
            # For capstone, extract the fish type and query directly
            fish_type = None
            for fish in ["King Salmon", "Sockeye Salmon", "Pacific Halibut",
                         "Yellowfin Tuna", "Dungeness Crab"]:
                if fish.lower() in customer_request.lower():
                    fish_type = fish
                    break
            if not fish_type:
                fish_type = "King Salmon"  # default

            catalog = query_vendor_catalog(fish_type)
            options = json.loads(catalog) if catalog.startswith('[') else []

            if not options:
                print(f"└─ No vendors available for {fish_type}!")
                return None

            print(f"│  {len(options)} vendors available")

            # ── Phase 3: Learning insights ─────────────────
            print(f"│")
            print(f"├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)")
            insights = learning.get_insights(fish_type)
            best_from_history = learning.best_vendor_for(fish_type)
            if best_from_history:
                print(f"│  Learning recommends: {best_from_history}")
                # Move preferred vendor to front
                options.sort(key=lambda x: (x['vendor'] != best_from_history,
                                            -x['freshness_score'], x['price_per_lb']))
            else:
                print(f"│  No history — first purchase of {fish_type}")

            # ── Phase 4: Try vendors with negotiation ───────
            print(f"│")
            print(f"├─ Phase 3: NEGOTIATION + GOALS + APPROVAL")

            # Extract qty from request (simple parse)
            qty = 5.0  # default
            for word in customer_request.split():
                try:
                    val = float(word)
                    if 0.5 <= val <= 50:
                        qty = val
                        break
                except ValueError:
                    continue

            deal = None
            for option in options:
                vendor = option["vendor"]
                max_price = min(option["price_per_lb"], budget / qty)

                print(f"│  Trying {vendor} (${option['price_per_lb']}/lb, "
                      f"freshness {option['freshness_score']}/10)...")

                # Quality gate (Reflection)
                if option["freshness_score"] < min_freshness:
                    print(f"│  ❌ Rejected by inspector: freshness {option['freshness_score']} < {min_freshness}")
                    exceptions.record("QUALITY_REJECTED", vendor,
                                      f"Freshness {option['freshness_score']} below minimum {min_freshness}")
                    continue

                try:
                    deal = await run_negotiation_and_approval(
                        vendor_name=vendor, fish_type=fish_type,
                        qty=qty, max_price_per_lb=max_price,
                        goals=goals, auto_approve=auto_approve)
                    if deal:
                        break
                except Exception as e:
                    exc_msg = exceptions.record(type(e).__name__, vendor, str(e))
                    print(f"│  ❌ {exc_msg}")

            # ── Final Summary ────────────────────────────
            print(f"│")
            if deal:
                print(f"└─ ✅ PIPELINE COMPLETE")
                print(f"   Purchased {deal['qty']}lb {deal['fish_type']} from {deal['vendor']}")
                print(f"   Total: ${deal['total']:.2f} (freshness: {deal['freshness']}/10)")
            else:
                print(f"└─ ❌ PIPELINE FAILED — no deal reached")
                print(f"   {exceptions.summary()}")

            tool_logger.summary()  # Show full tool call sequence
            return deal

        finally:
            # Restore original server
            if original_server:
                mcp_servers[fault_vendor] = original_server


print("CapstoneOrchestrator ready")

CapstoneOrchestrator ready


In [20]:
# ── Scenario 1: Happy Path (everything works) ─────────

# Reset for clean run
memory = PurchaseMemory()
learning = LearningEngine(memory)
exceptions = ExceptionHandler()

orchestrator = CapstoneOrchestrator()

deal1 = await orchestrator.run(
    customer_request=(
        "I need 5 lbs of fresh King Salmon for a dinner party this Saturday. "
        "Budget is $150. Needs to be sashimi-grade."
    ),
    budget=150,
    min_freshness=8,
)


############################################################
# CAPSTONE PIPELINE
# Request: I need 5 lbs of fresh King Salmon for a dinner party this Saturday. Budget is $150. Needs to be sashimi-grade.
# Budget: $150, Min Freshness: 8/10
############################################################

┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)
    🔧 [ 1] + 0.731s  get_learning_insights(fish_type='King Salmon')
    🔧 [ 2] + 1.441s  query_vendor_catalog(fish_type='King Salmon')
    🔧 [ 3] + 2.159s  check_freshness(vendor='Wild Salmon Co', fish='King Salmon')
    🔧 [ 4] + 2.789s  estimate_shipping(vendor='Wild Salmon Co', qty=5)
    🔧 [ 5] + 3.478s  check_freshness(vendor='Atlantic Fishery', fish='King Salmon')
│  Buyer found options
    🔧 [ 6] + 4.922s  query_vendor_catalog(fish_type='King Salmon')
│  2 vendors available
│
├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)
│  No history — first purchase of King Salmon
│
├─ Phase 3: NEGOTIATION + GOALS + APPROVAL
│  Trying Wild

In [21]:
# ── Scenario 2: Exception Recovery (Wild Salmon out of stock) ─

deal2 = await orchestrator.run(
    customer_request="5 lbs King Salmon, budget $150",
    budget=150,
    min_freshness=5,  # relaxed for fallback
    fault_vendor="Wild Salmon Co",
    fault_type="OUT_OF_STOCK",
)

# ── Scenario 3: Learning in Action (3rd purchase) ────────
# By now, memory has 2 purchases. The system should use
# learning insights to prefer the better vendor.

print("\n" + "=" * 60)
print("BEFORE RUN 3 — Learning State:")
print(memory.summary())
print(f"Best vendor for King Salmon: {learning.best_vendor_for('King Salmon')}")
print("=" * 60)

deal3 = await orchestrator.run(
    customer_request="5 lbs King Salmon, budget $150",
    budget=150,
    min_freshness=7,
)


############################################################
# CAPSTONE PIPELINE
# Request: 5 lbs King Salmon, budget $150
# Budget: $150, Min Freshness: 5/10
############################################################

⚠️  Fault injected: OUT_OF_STOCK at Wild Salmon Co

┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)
    🔧 [ 1] + 0.812s  get_learning_insights(fish_type='King Salmon')
    🔧 [ 2] + 1.530s  query_vendor_catalog(fish_type='King Salmon')
    🔧 [ 3] + 2.351s  check_freshness(vendor='Atlantic Fishery', fish='King Salmon')
    🔧 [ 4] + 3.167s  estimate_shipping(vendor='Atlantic Fishery', qty=5)
│  Buyer found options
    🔧 [ 5] + 4.488s  query_vendor_catalog(fish_type='King Salmon')
│  1 vendors available
│
├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)
│  Learning recommends: Wild Salmon Co
│
├─ Phase 3: NEGOTIATION + GOALS + APPROVAL
│  Trying Atlantic Fishery ($18/lb, freshness 6/10)...

NEGOTIATION: 5.0lb King Salmon from Atlantic Fishery
Max price: $18/lb (b

In [23]:
# ── Final State: What the System Learned ───────────────

print("=" * 60)
print("FINAL SYSTEM STATE")
print("=" * 60)

print(f"\n{memory.summary()}")
print(f"\n{learning.get_insights('King Salmon')}")
print(f"\n{exceptions.summary()}")

FINAL SYSTEM STATE

Purchase history: 3 transactions
  Wild Salmon Co: 2 purchases, reliability 0.90
  Atlantic Fishery: 1 purchases, reliability 0.60

Learning insights for King Salmon:
  Past purchases: 3
  Best vendor: Atlantic Fishery
  Average price: $22.82/lb
  Best freshness seen: 9/10

Exception log: 1 events
  [QUALITY_REJECTED] Atlantic Fishery: Freshness 6 below minimum 7


---
## Part 5: Real MCP — From Mock to Production

Same patterns from Part 4, now with:
- **FastMCP server** — real MCP protocol over stdio
- **ADK SqliteSessionService** — purchase history persists in SQLite across subprocess restarts
- **Lightweight client agents** — each agent is just an instruction + `McpToolset`

The server owns all state and business logic. The client stays truly lightweight.

### What Changes (Mock → Real)

| | Part 4 (Mock) | Part 5 (Real MCP) |
|---|---|---|
| **Tool calls** | `FunctionTool(query_vendor_catalog)` | `McpToolset(connection_params=...)` over stdio |
| **Memory** | `PurchaseMemory` class (in-memory) | `SqliteSessionService` (disk-persisted) |
| **Learning** | `LearningEngine` class (in-memory) | `run_learning_inference` MCP tool (server-side) |
| **Transport** | In-process function calls | JSON-RPC 2.0 over stdio pipe |
| **Persistence** | Lost when notebook restarts | Survives across subprocess restarts |

**What stays the same:** The patterns, the vendor data, the scenarios, and the outcomes.

### The Server Architecture

```
┌─ Client (this notebook) ──────────────────┐
│  BuyerAgent        → 1 instruction only   │
│  LearnerAgent      → 1 instruction only   │
│  NegotiatorAgent   → 1 instruction only   │
│  RecorderAgent     → 1 instruction only   │
│  MCPOrchestrator   → Python driver        │
│  (No memory, no state, just tool calls)   │
└────────────┬──────────────────────────────┘
             │ stdio (JSON-RPC 2.0)
             │ Each _call_agent() = new subprocess
             │ BUT: server state survives via SQLite!
             ▼
┌─ Server (pike_place_server.py) ───────────┐
│  FastMCP + ADK SqliteSessionService       │
│                                           │
│  Stateless tools (read VENDORS dict):     │
│    query_inventory, check_freshness       │
│    estimate_shipping, get_market_snapshot  │
│                                           │
│  Stateful tools (ADK session state):      │
│    record_purchase    → writes session    │
│    get_purchase_history → reads session   │
│    run_learning_inference → reads session │
│                                           │
│  Protocol tool (in-memory per process):   │
│    negotiate_price → NegotiationProtocol  │
│                                           │
│  ┌───────────────────────┐                │
│  │  pike_place_memory.db │ ← SQLite       │
│  │  (persists across     │                │
│  │   subprocess restarts)│                │
│  └───────────────────────┘                │
└───────────────────────────────────────────┘
```

In [24]:
%%writefile pike_place_server.py
#!/usr/bin/env python3
"""
Pike Place Fish Market — MCP Server

A real FastMCP server that exposes vendor operations as MCP tools.
Runs as a subprocess. ADK clients connect via stdio (JSON-RPC 2.0).

Server-side patterns:
  - A2A Negotiation Protocol (structured offer/counter/accept)
  - ADK SqliteSessionService for persistent purchase memory
  - Learning Engine (recommends best vendors from persisted data)
  - Exception Handling (logs failures, suggests recovery)

Persistence:
  Purchase history and vendor scores are stored in a SQLite database
  via ADK's SqliteSessionService. This means data survives across
  subprocess restarts — each new stdio connection picks up where the
  last one left off.

Tools exposed:
  1. query_inventory     — check vendor stock (stateless)
  2. check_freshness     — quality assessment (stateless)
  3. negotiate_price     — A2A negotiation (in-memory per process)
  4. estimate_shipping   — cost + delivery time (stateless)
  5. get_purchase_history — query persisted memory (ADK state)
  6. run_learning_inference — best vendor recommendation (ADK state)
  7. record_purchase     — save to persisted memory (ADK state)
"""

import json, os
from typing import Optional, List, Dict
from datetime import datetime
from fastmcp import FastMCP

# ADK persistence imports
from google.adk.sessions.sqlite_session_service import SqliteSessionService
from google.adk.events.event import Event
from google.adk.events.event_actions import EventActions


import sys as _sys
import time as _time

class ServerLogger:
    """Server-side tool call logger. Prints to stderr so it shows
    in the notebook (stdout is reserved for MCP JSON-RPC over stdio)."""
    def __init__(self):
        self._seq = 0
        self._start = _time.time()
    
    def log(self, tool_name: str, args_summary: str):
        self._seq += 1
        elapsed = round(_time.time() - self._start, 3)
        _sys.stderr.write(
            f"    [SERVER 🔧 {self._seq:2d}] +{elapsed:6.3f}s  {tool_name}({args_summary})\n")
        _sys.stderr.flush()

_slog = ServerLogger()

# ═══════════════════════════════════════════════════════════
# VENDOR DATA (same 5 Pike Place vendors from Capstone)
# ═══════════════════════════════════════════════════════════

VENDORS = {
    "Wild Salmon Co": {
        "specialty": ["King Salmon", "Sockeye Salmon", "Coho Salmon"],
        "inventory": {"King Salmon": 25, "Sockeye Salmon": 40, "Coho Salmon": 15},
        "price_per_lb": {"King Salmon": 28, "Sockeye Salmon": 22, "Coho Salmon": 19},
        "freshness_score": {"King Salmon": 9, "Sockeye Salmon": 10, "Coho Salmon": 8},
        "reliability": 0.95,
        "min_margin": 0.20,
        "cost_basis": {"King Salmon": 18, "Sockeye Salmon": 14, "Coho Salmon": 12},
    },
    "Atlantic Fishery": {
        "specialty": ["King Salmon", "Atlantic Salmon"],
        "inventory": {"King Salmon": 50, "Atlantic Salmon": 100},
        "price_per_lb": {"King Salmon": 18, "Atlantic Salmon": 14},
        "freshness_score": {"King Salmon": 6, "Atlantic Salmon": 7},
        "reliability": 0.90,
        "min_margin": 0.15,
        "cost_basis": {"King Salmon": 12, "Atlantic Salmon": 9},
    },
    "Halibut House": {
        "specialty": ["Pacific Halibut", "Lingcod"],
        "inventory": {"Pacific Halibut": 12, "Lingcod": 20},
        "price_per_lb": {"Pacific Halibut": 35, "Lingcod": 24},
        "freshness_score": {"Pacific Halibut": 9, "Lingcod": 8},
        "reliability": 0.85,
        "min_margin": 0.25,
        "cost_basis": {"Pacific Halibut": 22, "Lingcod": 15},
    },
    "Tuna King": {
        "specialty": ["Yellowfin Tuna", "Albacore Tuna"],
        "inventory": {"Yellowfin Tuna": 30, "Albacore Tuna": 45},
        "price_per_lb": {"Yellowfin Tuna": 22, "Albacore Tuna": 16},
        "freshness_score": {"Yellowfin Tuna": 8, "Albacore Tuna": 7},
        "reliability": 0.92,
        "min_margin": 0.18,
        "cost_basis": {"Yellowfin Tuna": 14, "Albacore Tuna": 10},
    },
    "Crab Cart": {
        "specialty": ["Dungeness Crab", "King Crab Legs"],
        "inventory": {"Dungeness Crab": 35, "King Crab Legs": 8},
        "price_per_lb": {"Dungeness Crab": 16, "King Crab Legs": 45},
        "freshness_score": {"Dungeness Crab": 9, "King Crab Legs": 7},
        "reliability": 0.88,
        "min_margin": 0.22,
        "cost_basis": {"Dungeness Crab": 10, "King Crab Legs": 30},
    },
}


# ═══════════════════════════════════════════════════════════
# ADK PERSISTED MEMORY (SqliteSessionService)
# ═══════════════════════════════════════════════════════════
#
# Instead of in-memory PurchaseMemory/LearningEngine classes,
# we use ADK's SqliteSessionService for disk-based persistence.
# Each subprocess reads/writes the same SQLite file, so data
# survives across subprocess restarts.

DB_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)),
                       "pike_place_memory.db")
APP_NAME = "pike_place_market"
USER_ID = "market_buyer"
SESSION_ID = "purchase_memory"

session_service = SqliteSessionService(db_path=DB_PATH)


async def _get_or_create_session():
    """Get existing memory session or create one with empty state."""
    session = await session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    if not session:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID,
            state={"purchase_history": [], "vendor_scores": {}})
    return session


async def _update_state(session, **state_delta):
    """Write state changes to SQLite via ADK Event + append_event."""
    event = Event(
        author="pike_place_server",
        invocation_id=f"inv_{datetime.now().isoformat()}",
        actions=EventActions(state_delta=state_delta))
    await session_service.append_event(session, event)


def _compute_memory_summary(history, vendor_scores):
    """Compute summary stats from persisted data."""
    vendors = {}
    for vendor, scores in vendor_scores.items():
        vendors[vendor] = {
            "purchases": len(scores),
            "avg_reliability": round(sum(scores) / len(scores), 3),
        }
    return {"total_purchases": len(history), "vendors": vendors}


def _compute_learning_insights(fish_type, history):
    """Compute learning recommendations from persisted history."""
    fish_history = [h for h in history if h["fish_type"] == fish_type]
    if not fish_history:
        return {
            "fish_type": fish_type,
            "status": "NO_HISTORY",
            "message": f"No past purchases for {fish_type}.",
        }
    # Best vendor by freshness-to-price ratio
    vendor_scores = {}
    for h in fish_history:
        v = h["vendor"]
        score = h["freshness"] / max(h["price_per_lb"], 1)
        vendor_scores.setdefault(v, []).append(score)
    avg_scores = {v: sum(s) / len(s) for v, s in vendor_scores.items()}
    best = max(avg_scores, key=avg_scores.get)
    avg_price = round(
        sum(h["price_per_lb"] for h in fish_history) / len(fish_history), 2)
    return {
        "fish_type": fish_type,
        "status": "HAS_HISTORY",
        "past_purchases": len(fish_history),
        "best_vendor": best,
        "average_price_per_lb": avg_price,
        "best_freshness_seen": max(h["freshness"] for h in fish_history),
        "persistence": "ADK SqliteSessionService",
    }


# ═══════════════════════════════════════════════════════════
# A2A NEGOTIATION PROTOCOL (in-memory per subprocess)
# ═══════════════════════════════════════════════════════════
#
# Negotiation state is intentionally NOT persisted — each
# negotiation is a fresh conversation within one subprocess.

class NegotiationProtocol:
    """Structured A2A protocol for buyer-vendor negotiation.

    Tracks rounds and the last counter price so that re-offering
    at or above the counter results in ACCEPTED (not another counter).
    """

    MAX_ROUNDS = 5  # server-enforced cap

    def __init__(self):
        self.rounds: List[Dict] = []
        self.last_counter: Optional[float] = None

    def reset(self):
        self.rounds = []
        self.last_counter = None

    @property
    def round_count(self) -> int:
        """Number of OFFER rounds (buyer submissions) so far."""
        return sum(1 for r in self.rounds if r["type"] == "OFFER")

    def add_round(self, msg_type: str, sender: str, **kwargs) -> Dict:
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": msg_type,
            "from": sender,
            "round": len(self.rounds) + 1,
            **kwargs,
        }
        self.rounds.append(msg)
        if msg_type == "COUNTER_OFFER":
            self.last_counter = kwargs.get("counter_price")
        return msg

    def trace(self) -> List[Dict]:
        return self.rounds.copy()


protocol = NegotiationProtocol()


# ═══════════════════════════════════════════════════════════
# FASTMCP SERVER + TOOLS
# ═══════════════════════════════════════════════════════════

mcp = FastMCP("Pike Place Vendor Server")


# ── Batch tool (reduces 10+ round-trips to 1) ────────────

@mcp.tool()
def get_market_snapshot(fish_type: str, qty_lbs: float = 1.0) -> str:
    _slog.log("get_market_snapshot", f"fish={fish_type}, qty={qty_lbs}")
    """Get a complete market snapshot for a fish type in ONE call.

    Returns inventory, freshness, quality grade, and shipping estimate
    for ALL vendors that carry this fish type. Use this instead of
    calling query_inventory + check_freshness + estimate_shipping
    individually for each vendor — it's 10x faster.

    Args:
        fish_type: Fish type to search for (e.g. 'King Salmon')
        qty_lbs: Quantity in pounds (for shipping estimate)
    """
    results = []
    for vendor_name, vendor in VENDORS.items():
        if fish_type not in vendor["inventory"]:
            continue
        freshness = vendor["freshness_score"][fish_type]
        quality = ("EXCELLENT" if freshness >= 9 else "GOOD" if freshness >= 7
                   else "FAIR" if freshness >= 5 else "POOR")
        shipping_cost = round(5 + qty_lbs * 1.0, 2)
        delivery_days = 1 if vendor_name in ["Wild Salmon Co", "Atlantic Fishery"] else 2
        results.append({
            "vendor": vendor_name,
            "fish_type": fish_type,
            "available_lbs": vendor["inventory"][fish_type],
            "price_per_lb": vendor["price_per_lb"][fish_type],
            "freshness_score": freshness,
            "quality_grade": quality,
            "sashimi_suitable": freshness >= 8,
            "reliability": vendor["reliability"],
            "shipping_cost": shipping_cost,
            "delivery_days": delivery_days,
        })
    if not results:
        return json.dumps({
            "fish_type": fish_type,
            "status": "NOT_FOUND",
            "message": f"No vendors carry {fish_type}",
            "all_vendors": list(VENDORS.keys()),
        })
    # Sort by freshness desc, then price asc
    results.sort(key=lambda r: (-r["freshness_score"], r["price_per_lb"]))
    return json.dumps({
        "fish_type": fish_type,
        "qty_lbs": qty_lbs,
        "vendors_found": len(results),
        "snapshot": results,
    })


# ── Stateless tools (read VENDORS dict only) ─────────────

@mcp.tool()
def query_inventory(vendor_name: str, fish_type: str) -> str:
    _slog.log("query_inventory", f"vendor={vendor_name}, fish={fish_type}")
    """Query a specific vendor's inventory for a fish type.

    Returns JSON with availability, price, freshness, and reliability.
    Use this to check what a vendor has in stock before buying.

    Args:
        vendor_name: Vendor name (e.g. 'Wild Salmon Co', 'Atlantic Fishery',
                     'Halibut House', 'Tuna King', 'Crab Cart')
        fish_type: Fish type (e.g. 'King Salmon', 'Pacific Halibut')
    """
    if vendor_name not in VENDORS:
        return json.dumps({"error": f"Unknown vendor: {vendor_name}",
                           "available_vendors": list(VENDORS.keys())})

    vendor = VENDORS[vendor_name]
    if fish_type not in vendor["inventory"]:
        return json.dumps({"vendor": vendor_name, "fish_type": fish_type,
                           "status": "NOT_CARRIED",
                           "carried_fish": vendor["specialty"]})

    return json.dumps({
        "vendor": vendor_name,
        "fish_type": fish_type,
        "available_lbs": vendor["inventory"][fish_type],
        "price_per_lb": vendor["price_per_lb"][fish_type],
        "freshness_score": vendor["freshness_score"][fish_type],
        "reliability": vendor["reliability"],
        "status": "AVAILABLE",
    })


@mcp.tool()
def check_freshness(vendor_name: str, fish_type: str) -> str:
    _slog.log("check_freshness", f"vendor={vendor_name}, fish={fish_type}")
    """Get detailed freshness assessment for a vendor's fish.

    Returns quality grade (EXCELLENT/GOOD/FAIR/POOR) and whether
    the fish is suitable for sashimi (requires freshness >= 8).

    Args:
        vendor_name: Vendor name
        fish_type: Fish type
    """
    inv = json.loads(query_inventory(vendor_name, fish_type))
    if "error" in inv or inv.get("status") != "AVAILABLE":
        return json.dumps({"error": f"{vendor_name} does not have {fish_type}",
                           "detail": inv})

    freshness = inv["freshness_score"]
    quality = ("EXCELLENT" if freshness >= 9 else "GOOD" if freshness >= 7
               else "FAIR" if freshness >= 5 else "POOR")

    return json.dumps({
        "vendor": vendor_name,
        "fish_type": fish_type,
        "freshness_score": freshness,
        "quality_grade": quality,
        "sashimi_suitable": freshness >= 8,
        "recommendation": ("Perfect for sashimi" if freshness >= 9
                           else "Acceptable for sashimi" if freshness >= 8
                           else "Cook only — not fresh enough for raw"),
    })


@mcp.tool()
def estimate_shipping(vendor_name: str, qty_lbs: float) -> str:
    _slog.log("estimate_shipping", f"vendor={vendor_name}, qty={qty_lbs}")
    """Estimate shipping cost and delivery time from a Pike Place vendor.

    All vendors use overnight cold pack shipping.
    Premium vendors (Wild Salmon Co, Atlantic Fishery) deliver in 1 day.
    Others deliver in 2 days.

    Args:
        vendor_name: Vendor name
        qty_lbs: Quantity in pounds
    """
    cost = round(5 + qty_lbs * 1.0, 2)
    days = 1 if vendor_name in ["Wild Salmon Co", "Atlantic Fishery"] else 2
    return json.dumps({
        "vendor": vendor_name,
        "shipping_cost": cost,
        "delivery_days": days,
        "method": "overnight cold pack",
    })


# ── A2A Negotiation (in-memory per subprocess) ───────────

@mcp.tool()
def negotiate_price(vendor_name: str, fish_type: str,
                    qty_lbs: float, offer_price_per_lb: float) -> str:
    _slog.log("negotiate_price", f"vendor={vendor_name}, fish={fish_type}, offer=${offer_price_per_lb}")
    """Run A2A negotiation protocol with a vendor.

    The server tracks negotiation state across calls. Each call is one round:
    1. Buyer sends OFFER at offer_price_per_lb
    2. Vendor evaluates and responds:
       - ACCEPTED: offer >= list price, or offer >= last counter, or max rounds hit
       - COUNTER_OFFER: offer above minimum but below acceptance threshold
       - REJECTED: offer below vendor's minimum price

    Max 5 rounds enforced server-side. To accept a counter-offer,
    call again with offer_price_per_lb >= the counter price.

    Args:
        vendor_name: Vendor to negotiate with
        fish_type: Fish type to buy
        qty_lbs: Quantity in pounds
        offer_price_per_lb: Your proposed price per pound
    """
    if vendor_name not in VENDORS:
        return json.dumps({"error": f"Unknown vendor: {vendor_name}"})

    vendor = VENDORS[vendor_name]
    if fish_type not in vendor["cost_basis"]:
        return json.dumps({"error": f"{vendor_name} does not carry {fish_type}"})

    cost = vendor["cost_basis"][fish_type]
    min_price = round(cost * (1 + vendor["min_margin"]), 2)
    list_price = vendor["price_per_lb"][fish_type]

    # Reset if: no prior rounds, different vendor/fish, or prior negotiation concluded
    concluded = any(r["type"] in ("ACCEPTED", "REJECTED") for r in protocol.rounds)
    is_new = (not protocol.rounds
              or concluded
              or protocol.rounds[0].get("vendor") != vendor_name
              or protocol.rounds[0].get("fish_type") != fish_type)
    if is_new:
        protocol.reset()

    # Record the buyer's offer
    protocol.add_round("OFFER", "BuyerAgent",
                       vendor=vendor_name, fish_type=fish_type,
                       qty_lbs=qty_lbs, offer_price=offer_price_per_lb)

    # ── Accept conditions ────────────────────────────
    # 1. Offer at or above list price → accept at list price
    # 2. Offer at or above the last counter → accept at counter price
    # 3. Max rounds reached and offer >= min → accept at offer price
    if offer_price_per_lb >= list_price:
        final_price = list_price
        protocol.add_round("ACCEPTED", vendor_name,
                           final_price=final_price, qty=qty_lbs)
        return json.dumps({
            "status": "ACCEPTED",
            "final_price_per_lb": final_price,
            "qty_lbs": qty_lbs,
            "total_cost": round(final_price * qty_lbs, 2),
            "a2a_trace": protocol.trace(),
        })

    if (protocol.last_counter is not None
            and offer_price_per_lb >= protocol.last_counter):
        final_price = protocol.last_counter
        protocol.add_round("ACCEPTED", vendor_name,
                           final_price=final_price, qty=qty_lbs,
                           reason="Offer meets previous counter")
        return json.dumps({
            "status": "ACCEPTED",
            "final_price_per_lb": final_price,
            "qty_lbs": qty_lbs,
            "total_cost": round(final_price * qty_lbs, 2),
            "a2a_trace": protocol.trace(),
        })

    if (protocol.round_count >= protocol.MAX_ROUNDS
            and offer_price_per_lb >= min_price):
        final_price = offer_price_per_lb
        protocol.add_round("ACCEPTED", vendor_name,
                           final_price=final_price, qty=qty_lbs,
                           reason=f"Max rounds ({protocol.MAX_ROUNDS}) reached")
        return json.dumps({
            "status": "ACCEPTED",
            "final_price_per_lb": final_price,
            "qty_lbs": qty_lbs,
            "total_cost": round(final_price * qty_lbs, 2),
            "a2a_trace": protocol.trace(),
        })

    # ── Counter or reject ────────────────────────────
    if offer_price_per_lb >= min_price:
        counter = round((offer_price_per_lb + list_price) / 2, 2)
        protocol.add_round("COUNTER_OFFER", vendor_name,
                           counter_price=counter, available_qty=qty_lbs)
        return json.dumps({
            "status": "COUNTER_OFFER",
            "counter_price_per_lb": counter,
            "available_qty": qty_lbs,
            "vendor_min_price": min_price,
            "rounds_used": protocol.round_count,
            "max_rounds": protocol.MAX_ROUNDS,
            "a2a_trace": protocol.trace(),
        })

    else:
        protocol.add_round("REJECTED", vendor_name,
                           reason=f"Offer ${offer_price_per_lb}/lb below minimum ${min_price}/lb")
        return json.dumps({
            "status": "REJECTED",
            "reason": f"Offer ${offer_price_per_lb}/lb is below vendor minimum ${min_price}/lb",
            "vendor_min_price": min_price,
            "a2a_trace": protocol.trace(),
        })


# ── Persisted tools (ADK SqliteSessionService) ───────────

@mcp.tool()
async def get_purchase_history(fish_type: str) -> str:
    _slog.log("get_purchase_history", f"fish={fish_type}")
    """Retrieve purchase history from ADK-persisted server memory.

    Returns past purchases for the specified fish type,
    including vendor, price, freshness, and timestamps.
    Data persists across server restarts via SqliteSessionService.

    Args:
        fish_type: Fish type to look up (e.g. 'King Salmon')
    """
    session = await _get_or_create_session()
    history = session.state.get("purchase_history", [])
    vendor_scores = session.state.get("vendor_scores", {})

    fish_history = [h for h in history if h["fish_type"] == fish_type]
    return json.dumps({
        "fish_type": fish_type,
        "count": len(fish_history),
        "records": fish_history[-5:],  # last 5
        "memory_summary": _compute_memory_summary(history, vendor_scores),
        "persistence": "ADK SqliteSessionService",
    })


@mcp.tool()
async def run_learning_inference(fish_type: str) -> str:
    _slog.log("run_learning_inference", f"fish={fish_type}")
    """Query the server-side learning engine for buying recommendations.

    Analyzes past purchases (persisted in ADK SqliteSessionService)
    to recommend the best vendor based on quality-to-price ratio.
    Also returns average price paid and best freshness score seen.

    Args:
        fish_type: Fish type to get recommendations for
    """
    session = await _get_or_create_session()
    history = session.state.get("purchase_history", [])
    return json.dumps(_compute_learning_insights(fish_type, history))


@mcp.tool()
async def record_purchase(vendor_name: str, fish_type: str, qty_lbs: float,
                          price_per_lb: float, freshness_score: int) -> str:
    _slog.log("record_purchase", f"vendor={vendor_name}, fish={fish_type}, qty={qty_lbs}, price=${price_per_lb}")
    """Record a completed purchase to ADK-persisted server memory.

    This updates vendor reliability scores and enables the
    learning engine to make better recommendations next time.
    Data persists via SqliteSessionService — survives server restarts.

    Args:
        vendor_name: Vendor purchased from
        fish_type: Fish type bought
        qty_lbs: Quantity in pounds
        price_per_lb: Final negotiated price per pound
        freshness_score: Freshness rating 1-10
    """
    session = await _get_or_create_session()
    history = session.state.get("purchase_history", [])
    vendor_scores = session.state.get("vendor_scores", {})

    # Build new record
    record = {
        "vendor": vendor_name, "fish_type": fish_type, "qty": qty_lbs,
        "price_per_lb": price_per_lb,
        "total_cost": round(qty_lbs * price_per_lb, 2),
        "freshness": freshness_score, "on_time": True,
        "timestamp": datetime.now().isoformat(),
    }
    history.append(record)

    # Update vendor reliability score
    score = (freshness_score / 10) * 1.0  # on_time=True
    vendor_scores.setdefault(vendor_name, []).append(score)

    # Persist to SQLite via ADK event
    await _update_state(session,
                        purchase_history=history,
                        vendor_scores=vendor_scores)

    return json.dumps({
        "status": "RECORDED",
        "record": record,
        "memory_state": _compute_memory_summary(history, vendor_scores),
        "persistence": "ADK SqliteSessionService",
    })


# ═══════════════════════════════════════════════════════════
# RUN SERVER
# ═══════════════════════════════════════════════════════════

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--sse", action="store_true",
                        help="Run as persistent SSE server (for orchestrator)")
    parser.add_argument("--port", type=int, default=8765,
                        help="Port for SSE server (default: 8765)")
    parser.add_argument("--host", default="localhost")
    args = parser.parse_args()

    if args.sse:
        mcp.run(transport="sse", host=args.host, port=args.port,
                show_banner=False)
    else:
        mcp.run()  # stdio (default for individual cells)


Overwriting pike_place_server.py


### Lightweight Client Agents

Each agent is minimal — just an instruction and `McpToolset`. No business logic, no state, no persistence. The LLM reasons about **what** to do; the server handles **how**.

Compare to Part 4 where agents had `FunctionTool` wrappers calling in-process functions. Here, `McpToolset` routes every call over stdio to the FastMCP server.

In [25]:
# ── Cell 4: Create ADK Client Agents with McpToolset ──────
#
# These agents are LIGHTWEIGHT — no vendor data, no memory,
# no negotiation logic. They just call MCP tools on the server.
#
# ─── HOW THE LLM KNOWS WHICH FUNCTIONS TO CALL ─────────────
#
# Notice: we never list individual tools like FunctionTool(query_vendor_catalog).
# Instead, each agent gets: tools=[McpToolset(connection_params=MCP_CONNECTION)]
#
# This is a CONNECTION, not a function list. Here's what happens at runtime
# when Runner.run_async() starts for any of these agents:
#
#   1. ADK calls McpToolset.connect()
#      → Spawns: python pike_place_server.py   (a child subprocess)
#      → Transport: stdio pipe (JSON-RPC 2.0 over stdin/stdout)
#
#   2. ADK sends "tools/list" over the stdio pipe
#      → This is the MCP discovery handshake — "what tools do you have?"
#      → The LLM has NOT been called yet. This is framework plumbing.
#
#   3. The FastMCP server responds with 8 tool schemas:
#      [
#        { "name": "get_market_snapshot",    "description": "Get a complete...",  "inputSchema": {...} },
#        { "name": "query_inventory",        "description": "Query a specific...", "inputSchema": {...} },
#        { "name": "check_freshness",        "description": "Get detailed...",    "inputSchema": {...} },
#        { "name": "estimate_shipping",      "description": "Estimate shipping.", "inputSchema": {...} },
#        { "name": "negotiate_price",        "description": "Run A2A negotiat.", "inputSchema": {...} },
#        { "name": "get_purchase_history",   "description": "Retrieve purchase.", "inputSchema": {...} },
#        { "name": "run_learning_inference", "description": "Query the learning", "inputSchema": {...} },
#        { "name": "record_purchase",        "description": "Record a completed", "inputSchema": {...} },
#      ]
#
#   4. ADK injects these 8 schemas into the LLM's context (as available tools).
#      Now Gemini's prompt looks like:
#        System: "You are a fish buyer..."
#        Available tools: [get_market_snapshot, query_inventory, ...]  ← from server
#        User: "Find all vendors with King Salmon..."
#
#   5. The LLM reads the tool DESCRIPTIONS and decides which to call.
#      It picks "get_market_snapshot" because the description says
#      "Get a complete market snapshot in ONE call" — that matches the task.
#
#   6. ADK converts the LLM's function_call into a JSON-RPC "tools/call"
#      message, sends it over stdio to the server, gets the response,
#      and streams it back as a function_response event.
#
# KEY INSIGHT: The LLM matched the tool by reading its description and
# JSON Schema — the same way you'd read API docs. The agent instruction
# helped ("Call get_market_snapshot..."), but the LLM could figure it out
# from the descriptions alone. This is MCP discovery in action.
#
# Compare with Part 4 (mock) where tools are hardcoded:
#   Part 4: tools=[FunctionTool(query_vendor_catalog), ...]  ← you decide at code time
#   Part 5: tools=[McpToolset(connection_params=...)]         ← server decides at runtime
# ────────────────────────────────────────────────────────────

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# Connection config: ADK will start pike_place_server.py as a subprocess
# and communicate over stdio (JSON-RPC 2.0)
MCP_CONNECTION = StdioConnectionParams(
    server_params=StdioServerParameters(
        command="python",
        args=["pike_place_server.py"],
    ),
    timeout=10.0,
)

# ── Buyer Agent ──────────────────────────────────────
# Discovers options by querying the MCP server
buyer_agent = LlmAgent(
    name="BuyerAgent",
    model=MODEL,
    instruction="""
You are a fish buyer at Pike Place Market. Given a customer request:

1. Call get_market_snapshot with the fish type to get ALL vendor data
   (inventory, freshness, shipping) in a single call.
2. If a snapshot isn't available, fall back to individual tool calls:
   query_inventory for each vendor, then check_freshness, then estimate_shipping.

Return a RANKED list of options with:
- Vendor name, price/lb, freshness score, quality grade, shipping cost
- Your recommendation for which vendor is best and why.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

# ── Learner Agent ────────────────────────────────────
# Focused agent: queries learning engine ONLY — no discovery
learner_agent = LlmAgent(
    name="LearnerAgent",
    model=MODEL,
    instruction="""
You query the learning engine for purchase history insights.

RULES:
1. Call run_learning_inference with the fish type provided.
2. Report EXACTLY what the learning engine returns — nothing more.
3. Do NOT call any other tools (no query_inventory, no check_freshness).
4. If the result says NO_HISTORY, say so clearly.
5. If the result says HAS_HISTORY, report: best vendor, average price,
   past purchases count, and best freshness seen.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

# ── Negotiator Agent ─────────────────────────────────
# Handles price negotiation via MCP server's A2A protocol
negotiator_agent = LlmAgent(
    name="NegotiatorAgent",
    model=MODEL,
    instruction="""
You are a tough but fair price negotiator for Pike Place Market.

STRATEGY — always try to get the best deal:
1. COMPUTE your opening offer: start 10-15% BELOW the target price.
   For example, if target is $28.00/lb, open around $23.80-$25.20/lb.
   Pick the exact number — this is YOUR strategic decision.
2. If you get a COUNTER_OFFER, DO NOT immediately accept it.
   Instead, offer a price BETWEEN your last offer and the counter
   (split the difference). This shows you're willing to negotiate.
3. If you get another COUNTER_OFFER, you may accept it by offering
   at or above the counter price — but only after at least 2 rounds
   of back-and-forth.
4. The server enforces a max of 5 rounds, so don't worry about
   going forever.

RULES:
- Never offer above your target price.
- Always call negotiate_price to make each offer — that's how
  the server tracks the A2A protocol.
- To ACCEPT a counter, call negotiate_price with offer >= the counter.
- Report the final deal with: price, total cost, rounds, and A2A trace.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

# ── Recorder Agent ───────────────────────────────────
# Records purchases to server-side memory
recorder_agent = LlmAgent(
    name="RecorderAgent",
    model=MODEL,
    instruction="""
You record completed purchases to the server's memory.

Given purchase details (vendor, fish type, quantity, price, freshness):
1. Call record_purchase with the exact details provided.
2. Confirm the recording was successful.
3. Report the updated memory state.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

print("Client agents created:")
print(f"  BuyerAgent      — discovers vendors via MCP (uses get_market_snapshot)")
print(f"  LearnerAgent    — queries learning engine ONLY (1 MCP call)")
print(f"  NegotiatorAgent — negotiates prices via A2A on server (up to 5 rounds)")
print(f"  RecorderAgent   — records purchases to server memory")
print(f"\nMCP connection: StdioConnectionParams → pike_place_server.py (timeout: 10s)")
print(f"Model: {MODEL} with auto-retry on 429s")

Client agents created:
  BuyerAgent      — discovers vendors via MCP (uses get_market_snapshot)
  LearnerAgent    — queries learning engine ONLY (1 MCP call)
  NegotiatorAgent — negotiates prices via A2A on server (up to 5 rounds)
  RecorderAgent   — records purchases to server memory

MCP connection: StdioConnectionParams → pike_place_server.py (timeout: 10s)
Model: gemini-2.0-flash with auto-retry on 429s


In [26]:
# ── Full Pipeline: MCPOrchestrator ────────────────────────
# Uses simple stdio connections. Each _call_agent() may start a
# new server subprocess, but purchase memory PERSISTS via ADK's
# SqliteSessionService (SQLite file on disk).

import os, re

MEMORY_DB = "pike_place_memory.db"

# Set to True  → fresh start every run (good for clean demos)
# Set to False → accumulate history across runs (watch learning improve!)
FRESH_START = True

if FRESH_START and os.path.exists(MEMORY_DB):
    os.remove(MEMORY_DB)
    print(f"  ♻ Reset {MEMORY_DB} (FRESH_START=True)")
elif os.path.exists(MEMORY_DB):
    import sqlite3, json as _json
    conn = sqlite3.connect(MEMORY_DB)
    row = conn.execute(
        "SELECT state FROM sessions WHERE id = ?", ("purchase_memory",)
    ).fetchone()
    conn.close()
    if row:
        n = len(_json.loads(row[0]).get("purchase_history", []))
        print(f"  📚 Keeping {MEMORY_DB} — {n} past purchases (FRESH_START=False)")
    else:
        print(f"  📚 Keeping {MEMORY_DB} (FRESH_START=False)")


def _parse_negotiated_price(negotiation_text: str, fallback: float) -> float:
    """Extract the final negotiated price from the agent's response.

    Handles Gemini escaped underscores (final_price variants) and various
    prose formats (Price: $26.82/lb, Price per lb: $26.43, etc.)
    """
    # Normalize: strip backslash-escapes that Gemini adds
    clean = negotiation_text.replace("\\_", "_")
    # Try A2A trace: 'final_price': 26.82
    match = re.search(r'["\'"]final_price["\'"]:\s*([\d.]+)', clean)
    if match:
        return float(match.group(1))
    # Try prose: "Price: $26.82/lb" or "Price per lb: $26.43"
    match = re.search(r"[Pp]rice(?:\s+per\s+lb)?[=:]\s*\$?([\d.]+)", clean)
    if match:
        return float(match.group(1))
    return fallback


class MCPOrchestrator:
    """Orchestrates the Pike Place buying pipeline over MCP.

    Client-side only — lightweight. All complex logic and
    persistence runs on the server (SqliteSessionService).

    Fixes applied:
    - Phase 2 uses LearnerAgent (1 call) not BuyerAgent (10 calls)
    - Phase 5 records actual negotiated price, not target price
    - Vendor selection influenced by learning data
    """

    async def _call_agent(self, agent: LlmAgent, message: str,
                          label: str = "AGENT") -> str:
        """Send a message to an agent and collect the full event stream.
        
        Logs every event in detail so you can trace:
          - Which agent is running and what message it received
          - Each MCP tool call (name + args) as the LLM makes it
          - Each tool response (truncated to 150 chars)
          - Whether calls were parallel (same event) or sequential
          - The LLM's text reasoning and final answer
          - Total tool calls, event count, and elapsed time
        """
        import time as _time
        _start = _time.time()
        
        session_service = InMemorySessionService()
        runner = Runner(
            agent=agent, app_name="mcp_pipeline",
            session_service=session_service)
        session = await session_service.create_session(
            app_name="mcp_pipeline", user_id="orchestrator")

        content = types.Content(
            role="user",
            parts=[types.Part.from_text(text=message)])

        # ── MCP Discovery: show tools/list handshake ─────────────
        # Before the LLM sees anything, ADK connects to the MCP server
        # via stdio and asks "what tools do you have?" (tools/list).
        # The server responds with tool schemas — THAT's what the LLM
        # uses to decide which function to call.
        print(f"\n  ┌──────────────────────────────────────────────────────")
        print(f"  │ AGENT: {agent.name} ({label})")
        try:
            # Probe the agent's toolset to show discovered MCP tools
            if hasattr(agent, '_tools') and agent._tools:
                for toolset in agent._tools:
                    if hasattr(toolset, '_tools') and toolset._tools:
                        tool_names = [t.name for t in toolset._tools if hasattr(t, 'name')]
                        if tool_names:
                            print(f"  │ 🔍 MCP Discovery: server exposed {len(tool_names)} tools via tools/list")
                            for tn in tool_names:
                                print(f"  │    → {tn}")
            elif hasattr(agent, 'tools') and agent.tools:
                for toolset in agent.tools:
                    if hasattr(toolset, 'get_tools'):
                        try:
                            tools = await toolset.get_tools()
                            if tools:
                                print(f"  │ 🔍 MCP Discovery: server exposed {len(tools)} tools via tools/list")
                                for t in tools:
                                    tname = t.name if hasattr(t, 'name') else str(t)
                                    print(f"  │    → {tname}")
                        except Exception:
                            pass  # Discovery logging is best-effort
        except Exception:
            pass  # Don't let logging break the pipeline
        print(f"  │ Message: {message[:100]}{'...' if len(message) > 100 else ''}")
        print(f"  │ Model: {agent.model}")
        print(f"  ├──────────────────────────────────────────────────────")

        response = ""
        tool_calls = 0
        event_count = 0
        calls_in_current_event = 0  # Track parallel calls within one event
        
        # ── ADK Event Stream ──────────────────────────────────────
        # runner.run_async() yields Event objects in real time.
        # Each event.content.parts[] contains one of:
        #   part.function_call     → LLM invoked an MCP tool (name + args)
        #   part.function_response → MCP server returned a result
        #   part.text              → LLM reasoning or final answer
        # is_final_response() = agent is done → break
        # ─────────────────────────────────────────────────────────
        async for event in runner.run_async(
            user_id="orchestrator", session_id=session.id,
            new_message=content
        ):
            event_count += 1
            elapsed = round(_time.time() - _start, 3)
            calls_in_current_event = 0
            
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "function_call") and part.function_call:
                        tool_calls += 1
                        calls_in_current_event += 1
                        name = part.function_call.name
                        args = dict(part.function_call.args) if part.function_call.args else {}
                        args_short = ", ".join(f"{k}={repr(v)[:30]}" for k, v in args.items())
                        parallel_tag = " ⚡PARALLEL" if calls_in_current_event > 1 else ""
                        print(f"  │ 🔧 [{tool_calls:2d}] +{elapsed:6.3f}s  {name}({args_short}){parallel_tag}")
                    elif hasattr(part, "function_response") and part.function_response:
                        result = str(part.function_response.response)[:150]
                        print(f"  │     ↳ Result: {result}...")
                    elif hasattr(part, "text") and part.text and part.text.strip():
                        response += part.text
                        # Show first 120 chars of LLM text
                        snippet = part.text.strip()[:120]
                        print(f"  │ 💬 LLM: {snippet}{'...' if len(part.text.strip()) > 120 else ''}")
            
            if event.is_final_response():
                # Dedup: grab any text from final event not already captured
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, "text") and part.text and part.text.strip():
                            if part.text not in response:
                                response += part.text
                elapsed = round(_time.time() - _start, 3)
                print(f"  │ ✅ FINAL (event #{event_count}, +{elapsed:.3f}s)")
                break

        elapsed = round(_time.time() - _start, 3)
        print(f"  ├──────────────────────────────────────────────────────")
        print(f"  │ Summary: {tool_calls} MCP calls, {event_count} events, {elapsed:.3f}s total")
        print(f"  └──────────────────────────────────────────────────────\n")

        return response.strip()

    async def run(self, fish_type: str, qty: float,
                  budget: float, target_price: float,
                  vendor: str = "Wild Salmon Co",
                  auto_approve: bool = True) -> dict:
        """Execute the full buying pipeline."""

        # Max chars to display per phase output (keeps logs readable)
        phase_limit = 300

        def truncate(text, limit=300):
            return text[:limit] + "..." if len(text) > limit else text

        import time as _time
        _pipeline_start = _time.time()
        
        print(f"\n{'#'*60}")
        print(f"# MCP PIPELINE")
        print(f"# {qty}lb {fish_type}, budget ${budget}, target ${target_price}/lb")
        print(f"# Vendor: {vendor} | Auto-approve: {auto_approve}")
        print(f"{'#'*60}")

        # ── Phase 1: Discovery ──────────────────────────
        _phase_start = _time.time()
        print(f"\n┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)")
        discovery = await self._call_agent(
            buyer_agent,
            f"Find all vendors with {fish_type}. Check freshness and shipping. "
            f"Budget is ${budget} for {qty}lb. Which vendor is best?",
            label="BUYER")
        print(f"│  Discovery complete ({round(_time.time() - _phase_start, 1)}s)")
        print(f"│  {truncate(discovery, phase_limit)}")

        # ── Phase 2: Learning (FIX: uses LearnerAgent) ──
        print(f"│")
        _phase_start = _time.time()
        print(f"├─ Phase 2: LEARNING (LearnerAgent → 1 MCP call)")
        insights = await self._call_agent(
            learner_agent,
            f"Check what we know about {fish_type}.",
            label="LEARN")
        print(f"│  {truncate(insights, phase_limit)}")

        # Act on learning — prove the system uses prior experience
        if "best_vendor" in insights.lower() or "HAS_HISTORY" in insights:
            print(f"│  🎯 Learning active — system is using prior purchases to guide decisions")
            # FIX 4: Let learning influence vendor selection
            # Extract recommended vendor from learning insights
            best_match = re.search(r"[Bb]est vendor[: ]+([ A-Za-z]+)", insights)
            if best_match:
                learned_vendor = best_match.group(1).strip()
                if learned_vendor != vendor:
                    print(f"│  🔄 Learning recommends {learned_vendor} "
                          f"(overriding default: {vendor})")
                    vendor = learned_vendor
                else:
                    print(f"│  ✓ Learning confirms {vendor} is the best choice")
        else:
            print(f"│  📝 No prior data yet — this will be the baseline purchase")

        # ── Phase 3: Negotiation ────────────────────────
        print(f"│")
        _phase_start = _time.time()
        print(f"├─ Phase 3: NEGOTIATION (A2A on server)")
        negotiation = await self._call_agent(
            negotiator_agent,
            f"Negotiate with {vendor} for {qty}lb {fish_type}. "
            f"Target price: ${target_price}/lb. "
            f"Decide your opening offer (aim 10-15%% below target). "
            f"Try up to 3 rounds.",
            label="NEGOTIATE")
        print(f"│  {truncate(negotiation, phase_limit)}")

        # FIX 2: Parse actual negotiated price from agent response
        actual_price = _parse_negotiated_price(negotiation, target_price)
        if actual_price != target_price:
            print(f"│  💰 Negotiated ${actual_price}/lb "
                  f"(saved ${round(target_price - actual_price, 2)}/lb)")

        # ── Human Approval Gate ─────────────────────────
        print(f"│")
        _phase_start = _time.time()
        print(f"├─ Phase 4: HUMAN APPROVAL")
        if auto_approve:
            print(f"│  ✅ Auto-approved (set auto_approve=False for manual)")
            approved = True
        else:
            response = input(f"│  Approve purchase from {vendor}? (yes/no): ")
            approved = response.strip().lower() in ["yes", "y"]

        if not approved:
            print(f"└─ ❌ REJECTED by customer")
            return {"status": "REJECTED"}

        # ── Phase 5: Record Purchase (FIX: uses actual price) ──
        print(f"│")
        _phase_start = _time.time()
        print(f"├─ Phase 5: RECORD (persist to ADK SqliteSessionService)")
        record = await self._call_agent(
            recorder_agent,
            f"Record purchase: vendor={vendor}, fish_type={fish_type}, "
            f"qty_lbs={qty}, price_per_lb={actual_price}, freshness_score=9",
            label="RECORD")
        print(f"│  ✓ Purchase persisted to SQLite via ADK")

        # ── Done ────────────────────────────────────────
        deal = {
            "status": "COMPLETE",
            "vendor": vendor,
            "fish_type": fish_type,
            "qty": qty,
            "price_per_lb": actual_price,
            "total": round(qty * actual_price, 2),
        }
        print(f"│")
        _total = round(_time.time() - _pipeline_start, 1)
        print(f"└─ ✅ PIPELINE COMPLETE ({_total}s total)")
        print(f"   {deal}")
        return deal


print("MCPOrchestrator ready (4 fixes applied)")
print("  ✓ Phase 2: LearnerAgent (1 call, not 10)")
print("  ✓ Phase 5: records actual negotiated price")
print("  ✓ Phase 1: BuyerAgent uses get_market_snapshot")
print("  ✓ Learning influences vendor selection")

  ♻ Reset pike_place_memory.db (FRESH_START=True)
MCPOrchestrator ready (4 fixes applied)
  ✓ Phase 2: LearnerAgent (1 call, not 10)
  ✓ Phase 5: records actual negotiated price
  ✓ Phase 1: BuyerAgent uses get_market_snapshot
  ✓ Learning influences vendor selection


In [27]:
# ── Scenario 1: Happy Path ────────────────────────────────

orchestrator = MCPOrchestrator()

deal1 = await orchestrator.run(
    fish_type="King Salmon",
    qty=5.0,
    budget=150.0,
    target_price=28.0,
    vendor="Wild Salmon Co",
)


############################################################
# MCP PIPELINE
# 5.0lb King Salmon, budget $150.0, target $28.0/lb
# Vendor: Wild Salmon Co | Auto-approve: True
############################################################

┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)

  ┌──────────────────────────────────────────────────────
  │ AGENT: BuyerAgent (BUYER)


/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/adk/features/_feature_decorator.py:72: UserWarning: [EXPERIMENTAL] feature FeatureName.BASE_AUTHENTICATED_TOOL is enabled.
  check_feature_enabled()


  │ 🔍 MCP Discovery: server exposed 8 tools via tools/list
  │    → get_market_snapshot
  │    → query_inventory
  │    → check_freshness
  │    → estimate_shipping
  │    → negotiate_price
  │    → get_purchase_history
  │    → run_learning_inference
  │    → record_purchase
  │ Message: Find all vendors with King Salmon. Check freshness and shipping. Budget is $150.0 for 5.0lb. Which v...
  │ Model: gemini-2.0-flash
  ├──────────────────────────────────────────────────────
  │ 🔧 [ 1] + 1.976s  get_market_snapshot(fish_type='King Salmon', qty_lbs=5)
  │     ↳ Result: {'content': [{'type': 'text', 'text': '{"fish_type": "King Salmon", "qty_lbs": 5.0, "vendors_found": 2, "snapshot": [{"vendor": "Wild Salmon Co", "fis...
  │ 💬 LLM: Here's a breakdown of the King Salmon options:

**Wild Salmon Co:**
*   Price: $28/lb
*   Freshness: 9 (Excellent)
*   S...
  │ ✅ FINAL (event #3, +3.570s)
  ├──────────────────────────────────────────────────────
  │ Summary: 1 MCP calls, 3 events, 3.570s tot

In [27]:
# ── Scenario 2: Learning Demo (2nd purchase) ──────────────
# The server subprocess restarts, but the SQLite database
# (pike_place_memory.db) persists Scenario 1's purchase.
# run_learning_inference WILL see the King Salmon purchase!

print("\nScenario 2: Second purchase — ADK SqliteSessionService preserves memory")
print("BuyerAgent will call run_learning_inference and see past purchases.\n")

deal2 = await orchestrator.run(
    fish_type="King Salmon",
    qty=3.0,
    budget=100.0,
    target_price=28.0,
    vendor="Wild Salmon Co",
)


Scenario 2: Second purchase — ADK SqliteSessionService preserves memory
BuyerAgent will call run_learning_inference and see past purchases.


############################################################
# MCP PIPELINE
# 3.0lb King Salmon, budget $100.0, target $28.0/lb
# Vendor: Wild Salmon Co | Auto-approve: True
############################################################

┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)

  ┌──────────────────────────────────────────────────────
  │ AGENT: BuyerAgent (BUYER)
  │ 🔍 MCP Discovery: server exposed 8 tools via tools/list
  │    → get_market_snapshot
  │    → query_inventory
  │    → check_freshness
  │    → estimate_shipping
  │    → negotiate_price
  │    → get_purchase_history
  │    → run_learning_inference
  │    → record_purchase
  │ Message: Find all vendors with King Salmon. Check freshness and shipping. Budget is $100.0 for 3.0lb. Which v...
  │ Model: gemini-2.0-flash
  ├──────────────────────────────────────────────────────
  │ 🔧 [ 1] 

In [28]:
# ── Scenario 3: Different fish, different vendor ──────────

deal3 = await orchestrator.run(
    fish_type="Dungeness Crab",
    qty=4.0,
    budget=80.0,
    target_price=16.0,
    vendor="Crab Cart",
)


############################################################
# MCP PIPELINE
# 4.0lb Dungeness Crab, budget $80.0, target $16.0/lb
# Vendor: Crab Cart | Auto-approve: True
############################################################

┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)

  ┌──────────────────────────────────────────────────────
  │ AGENT: BuyerAgent (BUYER)
  │ 🔍 MCP Discovery: server exposed 8 tools via tools/list
  │    → get_market_snapshot
  │    → query_inventory
  │    → check_freshness
  │    → estimate_shipping
  │    → negotiate_price
  │    → get_purchase_history
  │    → run_learning_inference
  │    → record_purchase
  │ Message: Find all vendors with Dungeness Crab. Check freshness and shipping. Budget is $80.0 for 4.0lb. Which...
  │ Model: gemini-2.0-flash
  ├──────────────────────────────────────────────────────
  │ 🔧 [ 1] + 0.813s  get_market_snapshot(qty_lbs=4, fish_type='Dungeness Crab')
  │     ↳ Result: {'content': [{'type': 'text', 'text': '{"fish_type": "Dung

In [29]:
# ── Verify Persistence: Inspect SQLite ────────────────
import sqlite3, os

db_path = "pike_place_memory.db"
if os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Show all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    print(f"Tables in {db_path}: {[t[0] for t in tables]}")
    
    # Show session state
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
        count = cursor.fetchone()[0]
        print(f"  {table[0]}: {count} rows")
    
    conn.close()
else:
    print(f"{db_path} not found — run the scenarios above first")

Tables in pike_place_memory.db: ['app_states', 'user_states', 'sessions', 'events']
  app_states: 0 rows
  user_states: 0 rows
  sessions: 1 rows
  events: 3 rows


---
## Part 6: Reflection — What We Built

You've now implemented **16 agentic patterns** in a single system, twice:
- **Part 4 (Mock):** Visible plumbing — every tool call, every reasoning step transparent
- **Part 5 (Real MCP):** Production architecture — FastMCP server, SqliteSessionService, lightweight clients

Same vendors. Same fish. Same prices. Same outcomes. **Different protocol layer.**

### Pattern Coverage Summary

| Pattern | Book Ch | Part 4 (Mock) | Part 5 (Real MCP) | Key Takeaway |
|---------|---------|---------------|-------------------|--------------|
| Routing | Ch 2 | Orchestrator extracts intent | Same | Python code, not LLM |
| Parallelization | Ch 3 | `query_vendor_catalog` | `get_market_snapshot` | Batch tool = 1 call instead of 10 |
| Tool Use | Ch 5 | `FunctionTool` wrappers | `McpToolset` over stdio | Same interface, different transport |
| Reflection | Ch 6 | QualityInspector agent | Same | Agent rejects low-quality options |
| Multi-Agent | Ch 7 | Buyer, Inspector, Negotiator | Buyer, Learner, Negotiator, Recorder | Focused agents > general-purpose |
| Sequential | Ch 7 | CapstoneOrchestrator | MCPOrchestrator | Python drives the loop |
| AgentTool | Ch 7 | Mock server as tool | FastMCP as tool | MCP makes this natural |
| ReAct | Ch 8a | BuyerAgent iterates | Same | LLM reasons between calls |
| Plan-and-Execute | Ch 8b | Python orchestrator | Same | Deterministic driver, not LLM |
| Memory | New | PurchaseMemory class | SqliteSessionService | Server-side = survives restarts |
| Learning | New | LearningEngine class | `run_learning_inference` tool | Logic migrates with data |
| Goal Tracking | New | GoalTracker (client) | GoalTracker (client) | Stays on client = orchestration concern |
| Exception Handling | New | ExceptionHandler (client) | ExceptionHandler (client) | Client handles retry logic |
| A2A Protocol | New | NegotiationProtocol | `negotiate_price` tool | Protocol on server in production |
| MCP Server | New | MockMCPVendorServer | FastMCP + stdio | Mock → real, same interface |
| Human-in-the-Loop | New | auto_approve flag | Same | Gate before irreversible action |

### Thin Client vs Thick Server: The Tradeoff

| | Part 4 (Mock = Client-Heavy) | Part 5 (Real MCP = Server-Heavy) |
|---|---|---|
| **Debugging** | Easy — everything in one process | Harder — need to trace across stdio boundary |
| **Persistence** | None — lost on restart | SQLite survives restarts |
| **Scalability** | Single user, single process | Multiple clients can connect to same server |
| **LLM token usage** | Client LLM reasons about tool sequence | Same (client still orchestrates) |
| **Deployment** | One notebook | Server process + client notebook |

**The middle ground (production pattern):**

Expose **both** granular tools AND compound "task" tools from the same server:
- `get_market_snapshot` — granular, for clients that want control
- `execute_purchase_pipeline(fish_type, qty, budget)` — compound, does everything in one call

The client picks its level of delegation. Start granular for learning, graduate to compound for production.

### Notebook vs Production

| Component | This Notebook | Production System |
|-----------|--------------|------------------|
| Vendor Catalog | Mock dict with fixed data | Live vendor APIs or databases |
| MCP Server | FastMCP over stdio (local) | Containerized service over HTTP/SSE |
| Learning | Simple best-of averages | ML recommendation engine |
| Goal Tracking | Python class with thresholds | Constraint solver (OR-Tools) |
| Human Approval | `auto_approve` flag | Webhook + email/SMS notification |
| Memory | SQLite (local file) | PostgreSQL / Redis / data warehouse |
| Exception Handling | Fallback to next vendor | Retry + alerting + on-call escalation |
| Negotiation | Deterministic midpoint math | LLM-powered vendor agent (A2A) |

### Next Steps: The A2A Upgrade

This notebook leaves **one gap intentionally open**: true Agent-to-Agent communication.

Our negotiation is currently MCP — the client calls `negotiate_price`, the server applies deterministic math. To make it A2A:

1. **Give vendors their own LLMs.** Each vendor stall becomes an autonomous agent that reasons about pricing strategy, inventory levels, and customer history.

2. **Replace the tool call with a conversation.** Instead of `negotiate_price(offer=23.80)` → `{counter: 25.90}`, the buyer agent and vendor agent have a multi-turn dialogue:
   - Buyer: "I'll take 5lb at $23.80 — I'm a repeat customer"
   - Vendor: "I remember you. $25 for loyalty, but only if you take 6lb"
   - Buyer: "Deal at 5lb for $25?"
   - Vendor: "Accepted."

3. **Use Agent Cards for discovery.** Each vendor publishes what it can do. The buyer's orchestrator discovers available vendors at runtime — no hardcoded vendor list.

That's the natural next chapter. Same market, same vendors, same fish — different protocol layer.

**The core lesson from this entire notebook:**

> The pattern you wrap around an LLM determines how it thinks. Same Gemini 2.0 Flash model powers every agent here. But the Buyer reasons differently from the Inspector, which reasons differently from the Negotiator — because each has a different architecture, different tools, and different constraints. **Intelligence emerges from the system design, not just the model.**